In [1]:
import pandas as pd
import numpy as np
import csv
import re
from bs4 import BeautifulSoup
import camelot 

from tabula import read_pdf
from tabulate import tabulate
from itertools import chain
import matplotlib as plt

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

# %matplotlib inline
%config InlineBackend.figure_format='retina'
# If encounter SSL certify error - go to your Python 3.x.x folder and install Certification 

In [2]:
csv = pd.read_csv('farm_smoke.csv')
sonoma_data = csv.query('County == "Sonoma" and Year >= 2001').copy().reset_index(drop=True)
sonoma_data

,County,Year,Links
0,Sonoma,2021,https://sonomacounty.ca.gov//Main%20County%20S...
1,Sonoma,2020,https://sonomacounty.ca.gov//Main%20County%20S...
2,Sonoma,2019,https://sonomacounty.ca.gov//Main%20County%20S...
3,Sonoma,2018,https://sonomacounty.ca.gov//Main%20County%20S...
4,Sonoma,2017,https://sonomacounty.ca.gov//Main%20County%20S...
5,Sonoma,2016,https://sonomacounty.ca.gov//Main%20County%20S...
6,Sonoma,2015,https://sonomacounty.ca.gov//Main%20County%20S...
7,Sonoma,2014,https://sonomacounty.ca.gov//Main%20County%20S...
8,Sonoma,2013,https://sonomacounty.ca.gov//Main%20County%20S...
9,Sonoma,2012,https://sonomacounty.ca.gov//Main%20County%20S...


In [3]:
urls = sonoma_data['Links']
urls

0     https://sonomacounty.ca.gov//Main%20County%20S...
1     https://sonomacounty.ca.gov//Main%20County%20S...
2     https://sonomacounty.ca.gov//Main%20County%20S...
3     https://sonomacounty.ca.gov//Main%20County%20S...
4     https://sonomacounty.ca.gov//Main%20County%20S...
5     https://sonomacounty.ca.gov//Main%20County%20S...
6     https://sonomacounty.ca.gov//Main%20County%20S...
7     https://sonomacounty.ca.gov//Main%20County%20S...
8     https://sonomacounty.ca.gov//Main%20County%20S...
9     https://sonomacounty.ca.gov//Main%20County%20S...
10    https://sonomacounty.ca.gov//Main%20County%20S...
11    https://sonomacounty.ca.gov//Main%20County%20S...
12    https://sonomacounty.ca.gov//Main%20County%20S...
13    https://sonomacounty.ca.gov//Main%20County%20S...
14    https://sonomacounty.ca.gov//Main%20County%20S...
15    https://sonomacounty.ca.gov//Main%20County%20S...
16    https://sonomacounty.ca.gov//Main%20County%20S...
17    https://sonomacounty.ca.gov//Main%20County

In [4]:
# Note 1: 2021 - 2017 can work together 
# Note 2: 2016 - format diff, have -- and ' .' formating, top 12 reds 
# Note 3: 2015, 2014 - top 12 reds
# Note 4: 2013, 2012 - format change, words and style is diff; need to change camelot

table_list = []
for link, year in zip(urls, sonoma_data['Year']):
    page = '12'
    if year == 2017 or year == 2004 or year == 2003 or year == 2001:
        page = '10'
    elif year == 2007:
        page = '9'
    elif year == 2016 or year == 2013 or year == 2011 or year == 2010 or year == 2009:
        page = '8'
    elif year == 2014 or year == 2012 or year == 2006:
        page = '7'
    elif year == 2008 or year == 2005:
        page = '11'
    # utilize camelot to read pdf file at page 12, 
    # capture precisely with stream and close up the row gap with row_tol = 10

    if year <= 2011:
        if year == 2011:
            # specific reads for 2011
            tables = camelot.read_pdf(link, pages=page, table_areas=['30,720,600,250'], flavor='stream', row_tol=8)[0].df
        elif year == 2004:
            tables = camelot.read_pdf(link, pages=page, table_areas=['60,740,600,370'], flavor='stream', row_tol=8,split_text=True)[0].df
        elif year == 2005:
            tables = camelot.read_pdf(link, pages=page, table_areas=['50,720,600,300'], flavor='stream', row_tol=8, split_text=True)[0].df
        elif year <= 2010 and year != 2005 and year != 2004:
            tables = camelot.read_pdf(link, pages=page, flavor='stream', row_tol=8, split_text=True)[0].df
    elif year != 2012 and year != 2013:
        tables = camelot.read_pdf(link, pages=page, flavor='stream', row_tol=10)[0].df
    
    # display(tables)
    table_list.append(tables)
print('finish reading pdf files')


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 2
  warnings.warn(f"No tables found in table area {table_idx + 1}")
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 2
  warnings.warn(f"No tables found in table area {table_idx + 1}")
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 2
  warnings.warn(f"No tables found in table area {table_idx + 1}")


finish reading pdf files


In [5]:
def rename_replace_varietal_column(table, year):

    if year == 2016:
        table.columns = ['0', '1', '2', '3', '4', '5', '6', '7']
        # For 2016 crop report:
        # temp is non-bearing
        table['Temp'] = table['3']  # Create a temporary column and store 'Bearing' values
        table['3'] = table['2']  # Move 'Non-Bearing' values to 'Bearing' column
        table['2'] = table['Temp']  # Move temporary values to 'Non-Bearing' column
        table.drop('Temp', axis=1, inplace=True)  # Drop the temporary column
        
    elif year <= 2010:
        if year != 2006 and year != 2004 and year != 2003 and year != 2001:
            table.columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
            table['7'] = table['6'] + table['7']
            table['9'] = table['8'] + table['9']
            table = table.drop(['6', '8'], axis=1)
        elif year == 2006:
            table.columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8']
            table['7'] = table['6'] + table['7']
            table = table.drop(['6'], axis=1)

    new_columns = ['Varietal', 'Year', 'Bearing' , 'Non-Bearing', 'Total', 'Tons', '$/Ton', 'Total Value']
    data = table.rename(columns=dict(zip(table.columns,new_columns)))

    if year == 2011:
        # 2011 specific fixes
        # 24		2011	5,216.8	81.1	5,297.9	13,101.0	$ 2,357.61	$\n30,887,100
        # 25	Zinfandel
        data.loc[24][0] = data.loc[25][0]
        data.loc[27][0] = data.loc[28][0]
        data.loc[28][0] = ''

    elif year <= 2010 and year != 2007 and year != 2005:
        # for 2010; similar for 2009
        # 28		2010	37,804.4	1,606.3	39,410.7	108,481.0\n(a	) $2,255.85 (a	) $244,717,100
        # 29	TOTAL REDS	2009	37,937.2	3,636.2	41,573.4	120,717.0\n(a	) $2,430.05 (a	) $293,348,400	
        # 	
        if year == 2003 or year == 2002 or year == 2001:
            # specific for 2003, 2002, 2001: drop the total all wine grapes row
            data = data[:28]
        data.loc[28][0] = data.loc[29][0]
        data.loc[29][0] = ''
        
    elif year == 2007:
        data.loc[29][0] = data.loc[30][0]
        data.loc[30][0] = ''

    # data drop empty row
    data = data.query("Year != '' and Bearing != '' and Total != '' and Tons != ''")

    data['Varietal'] = data['Varietal'].replace('Mataro/', 'Mataro/Mouvedere')
    data['Varietal'] = data['Varietal'].replace('Cabernet', 'Cabernet Sauvignon')

    # for 2016 only; ignored when dataset does not contain these strings
     # for 2016; Sangiovese/Sangiove to Sangiovese 
    data['Varietal'] = data['Varietal'].replace('Sangiovese/', 'Sangiovese')
    # for 2016 Syrah - Shiraz to Syrah-Shiraz ; 
    data['Varietal'] = data['Varietal'].replace('Syrah - Shiraz', 'Syrah-Shiraz')

    # <= 2010; TOTAL REDS does not include Other Reds; 
    # > 2010; TOTAL ALL REDS include Other Reds
    # 2012, 2011 Only has TOTAL REDS 
    # Starting from <= 2003, Total Reds with Other Reds

    # data['Varietal'] = data['Varietal'].replace('TOTAL ALL REDS', 'TOTAL ALL REDS ')

    if year <= 2010:
        data['Varietal'] = data['Varietal'].replace('Total Reds', 'TOTAL ALL REDS NOT including Other Reds')
        data['Varietal'] = data['Varietal'].replace('TOTAL REDS', 'TOTAL ALL REDS NOT including Other Reds')

    # For x<=2016 and x>2010; total count of all reds is label as TOTAL REDS
    data['Varietal'] = data['Varietal'].replace('TOTAL REDS', 'TOTAL ALL REDS Includes Other Reds')

    # For >= 2017, total count of all reds is label as TOTAL ALL REDS
    data['Varietal'] = data['Varietal'].replace('TOTAL ALL REDS', 'TOTAL ALL REDS Includes Other Reds')

    # for 2005 and 2004 
    data['Varietal'] = data['Varietal'].replace('Cabernet Sauv.', 'Cabernet Sauvigno')
    # for 2004
    data['Varietal'] = data['Varietal'].replace('Syrah-shiraz', 'Syrah-Shiraz')

    # added Mourvedere (a), Sangioveto bc of 2016
    data['Varietal'] = data['Varietal'].replace(['', 'including other reds', 'Mouvedere', 'Mourvedere (a)', 'Sauvignon', 'Sangioveto'], np.nan)
    data = data.reset_index(drop=True)

    # duplicate each variable twice because there is empty space after each variable 
    # For example: Zinfandel 2015 2014 to Zinfandel 2015 Zinfandel 2014
    data['Varietal'] = data['Varietal'].dropna().repeat(2).reset_index(drop=True)

    return data

def add_shift_county_color_column(data):

    # add columns to last and pop
    data['County'] = 'Sonoma'
    first_column = data.pop('County')
    data['Grape Color'] = 'Red'
    third_column = data.pop('Grape Color')

    # put county as first column, grape color as 3rd
    data.insert(0, 'County', first_column)
    data.insert(2, 'Grape Color', third_column)
    return data

def fix_incorrect_digits(data, year):
    # if exist a string that has empty space, add a '.' to complete the number
    
    if year == 2016:
        data['Bearing'] = data['Bearing'].str.replace(' .', '.', regex=False)
        data['Bearing'] = data['Bearing'].str.replace('--', '0.0', regex=False)
        data['Non-Bearing'] = data['Non-Bearing'].str.replace(' .', '.', regex=False)
        data['Non-Bearing'] = data['Non-Bearing'].str.replace('--', '0.0', regex=False)

        data['Total'] = data['Total'].str.replace(' .', '.', regex=False)
        data['Total'] = data['Total'].str.replace('--', '0.0', regex=False)
        data['Tons'] = data['Tons'].str.replace(' ', '', regex=False)
        data['$/Ton'] = data['$/Ton'].str.replace(' .', '.', regex=False)
        data['Total Value'] = data['Total Value'].str.replace(' ', ',')
    elif year == 2015:
        data['$/Ton'] = data['$/Ton'].replace('$2.525.16', '$2,525.16')
        data['$/Ton'] = data['$/Ton'].replace('$3,525,42', '$3,525.42')
        data['Bearing'] = data['Bearing'].replace('11 689.7', '11,689.7')
    elif year == 2011:
        # for 2011 only 
        data['Bearing'] = data['Bearing'].str.replace(' ', '.')
        data['Non-Bearing'] = data['Non-Bearing'].str.replace(' ', '.')
        data['Total'] = data['Total'].str.replace(' ', '.')
        data['Tons'] = data['Tons'].str.replace(' ', '.')
        data['$/Ton'] = data['$/Ton'].str.replace(' ', '')
        # data['Total Value'] = data['Total Value'].str.replace(' ', ',')

        # specific for 2011
        data['Bearing'] = data['Bearing'].str.replace('*', '', regex=False)
        data['Tons'] = data['Tons'].str.replace('*', '', regex=False)
        data['$/Ton'] = data['$/Ton'].str.replace('*', '', regex=False)
        data['Total Value'] = data['Total Value'].str.replace('\n', '', regex=False)
        data['Total Value'] = data['Total Value'].str.replace('(a)  $ ', '$', regex=False)
        data['Total Value'] = data['Total Value'].str.replace('*', '', regex=False)  
    elif year <= 2010:
        data['Bearing'] = data['Bearing'].str.replace(' ', '.')
        data['Non-Bearing'] = data['Non-Bearing'].str.replace(' ', '.')
        data['Total'] = data['Total'].str.replace(' ', '.')

        # # specific for 2010, 2009, 2008
        data['Tons'] = data['Tons'].str.replace('\n(a', '', regex=False)
        
        data['$/Ton'] = data['$/Ton'].str.replace(') ', '', regex=False)
        data['$/Ton'] = data['$/Ton'].str.replace(' (a', '', regex=False)
        data['$/Ton']  = data['$/Ton'].replace('', '$0.0', regex=False)
        
        # specific for 2007
        data['$/Ton'] = data['$/Ton'].str.replace(' (a)', '', regex=False)
        data['$/Ton'] = data['$/Ton'].str.replace(')', '', regex=False)
        data['Total Value'] = data['Total Value'].str.replace(') ', '', regex=False)
        data['Total Value'] = data['Total Value'].str.replace('5,240.200', '5,240,200', regex=False)

        # specific for 2006:
        data['Bearing'] = data['Bearing'].str.replace('*', '', regex=False)
        data['Total'] = data['Total'].str.replace('*', '', regex=False)
        data['$/Ton'] = data['$/Ton'].str.replace(' (a)', '', regex=False)
        data['Total Value'] = data['Total Value'].str.replace(' \n', '', regex=False)
        data['Total Value'] = data['Total Value'].str.replace(' ', '', regex=False)

        # specific for 2004:
        data['$/Ton'] = data['$/Ton'].str.replace(' \n', '', regex=False)
        data['$/Ton'] = data['$/Ton'].str.replace(' ', '', regex=False)
    
        data['Total Value'] = data['Total Value'].apply(lambda x: '$' + x if '$' not in x else x)   

        # specific for 2003 and <= 2002 
        if year == 2003:
            data['Bearing'] = data['Bearing'] + '.0'
            data['Non-Bearing'] = data['Non-Bearing'] + '.0'
            data['Total'] = data['Total'] + '.0'
            data['Tons'] = data['Tons'].str.rstrip('0').replace('19,571.', '19,571.0')  
            
        elif year <= 2002:
            data['Bearing'] = data['Bearing'] + '.0'
            data['Non-Bearing'] = data['Non-Bearing'] + '.0'
            data['Total'] = data['Total'] + '.0'
            data['Tons'] = data['Tons'] + '.0'

    else:
        data['Bearing'] = data['Bearing'].str.replace(' ', '.')
        data['Non-Bearing'] = data['Non-Bearing'].str.replace(' ', '.')
        data['Total'] = data['Total'].str.replace(' ', '.')
        data['Tons'] = data['Tons'].str.replace(' ', '.')
        data['$/Ton'] = data['$/Ton'].str.replace(' ', '.')
        data['Total Value'] = data['Total Value'].str.replace(' ', ',')
    return data
        

In [6]:
# for <= 2016 we need to do it separately because of formatting 

combined_data_up_to_2017 = pd.DataFrame()
combined_data_up_to_2001 = pd.DataFrame()

# this goes up to 2017 files
for table, year in zip(table_list, sonoma_data['Year']):
    # 2021 - 2017 has a pattern where first 3 rows didn't matter
    
    if year >= 2017:
        table.drop([0,1,2], axis=0, inplace=True)

        # rename column names
        data_varietal_replaced = rename_replace_varietal_column(table, year)
        data_shifted = add_shift_county_color_column(data_varietal_replaced)

        # if exist a string that has empty space, add a '.' to complete the number
        data = fix_incorrect_digits(data_shifted, year)

        print(f'number of row for year {year} table is: {len(data)}')

        combined_data_up_to_2017 = pd.concat([combined_data_up_to_2017, data])

        # display(data)
        # display(combined_data_up_to_2017)
        
    # skips 2012 and 2013, their PDFs are not extractable
    elif year <= 2016 and year != 2013 and year != 2012:
        if year == 2009 or year == 2007 or year == 2003 or year == 2002 or year == 2001:
            table.drop([0,1,2], axis=0, inplace=True)
        else:
            table.drop([0,1], axis=0, inplace=True)
            
        # rename column names
        data_varietal_replaced = rename_replace_varietal_column(table, year)
        data_shifted = add_shift_county_color_column(data_varietal_replaced)

        # if exist a string that has empty space, add a '.' to complete the number
        data = fix_incorrect_digits(data_shifted, year)

        print(f'number of row for year {year} table is: {len(data)}')
        # display(data)

        combined_data_up_to_2001 = pd.concat([combined_data_up_to_2001, data])

number of row for year 2021 table is: 28
number of row for year 2020 table is: 28
number of row for year 2019 table is: 28
number of row for year 2018 table is: 28
number of row for year 2017 table is: 28
number of row for year 2016 table is: 26
number of row for year 2015 table is: 26
number of row for year 2014 table is: 26
number of row for year 2011 table is: 26
number of row for year 2010 table is: 28
number of row for year 2009 table is: 28
number of row for year 2008 table is: 28
number of row for year 2007 table is: 28
number of row for year 2006 table is: 28
number of row for year 2005 table is: 28
number of row for year 2004 table is: 28
number of row for year 2003 table is: 28
number of row for year 2002 table is: 28
number of row for year 2001 table is: 28


In [7]:
sonoma_2017 = combined_data_up_to_2017.drop_duplicates().reset_index(drop=True)
sonoma_2001 = combined_data_up_to_2001.drop_duplicates().reset_index(drop=True)
sonoma_2001

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Pinot Noir,Red,2016,"13,343.1",385.2,"13,728.3","42,566.3","$3,680.50","$156,665,300"
1,Sonoma,Pinot Noir,Red,2015,"12,850.8",750.0,"13,600.8","30,667.2","$3,525.42","$108,114,800"
2,Sonoma,Cabernet Sauvignon,Red,2016,"12,456.7",560.0,"13,016.7","41,575.7","$2,964.60","$123,255,300"
3,Sonoma,Cabernet Sauvignon,Red,2015,"11,808.8","1,014.7","12,823.5","32,947.3","$2,724.17","$89,754,100"
4,Sonoma,Zinfandel,Red,2016,"5,070.5",20.0,"5,090.5","15,287.7","$2,840.06","$43,418,000"
...,...,...,...,...,...,...,...,...,...,...
258,Sonoma,Sangiovese,Red,2000,281.0,68.0,349.0,"1,791.0","$1,859.57","$3,331,000"
259,Sonoma,Syrah-Shiraz,Red,2000,559.0,371.0,930.0,"3,349.0","$2,171.65","$7,272,000"
260,Sonoma,Zinfandel,Red,2000,"4,478.0",665.0,"5,143.0","13,223.0","$2,143.10","$28,337,800"
261,Sonoma,Other Reds,Red,2000,455.0,319.0,774.0,"1,844.0","$2,036.69","$3,755,200"


In [8]:
sonoma_2001_sorted = sonoma_2001.sort_values(['Year', 'Varietal'])
sonoma_2001_sorted = sonoma_2001_sorted.reset_index(drop=True)
sonoma_2001_sorted

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,"2,097.0","$2,226.53","$4,668,600"
1,Sonoma,Cabernet Sauvigno,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,417.96","$79,240,400"
2,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,"$1,391.55","$1,065,400"
3,Sonoma,Merlot,Red,2000,"6,564.0","1,058.0","7,622.0","31,480.0","$2,019.29","$63,566,200"
4,Sonoma,Meunier,Red,2000,129.0,17.0,146.0,637.0,"$2,069.27","$1,318,700"
...,...,...,...,...,...,...,...,...,...,...
258,Sonoma,Pinot Noir,Red,2016,"13,343.1",385.2,"13,728.3","42,566.3","$3,680.50","$156,665,300"
259,Sonoma,Sangiovese,Red,2016,316.9,0.0,316.9,"1,091.1","$2,314.48","$2,525,300"
260,Sonoma,Syrah-Shiraz,Red,2016,"1,575.6",13.5,"1,589.1","3,618.1","$2,596.01","$9,392,600"
261,Sonoma,TOTAL ALL REDS Includes Other Reds,Red,2016,"39,821.4","1,342.0","41,163.5","130,589.3","$3,008.75","$392,910,600"


In [9]:
sonoma_2001_sorted_copy = sonoma_2001_sorted.copy()
sonoma_2001_sorted_copy

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,"2,097.0","$2,226.53","$4,668,600"
1,Sonoma,Cabernet Sauvigno,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,417.96","$79,240,400"
2,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,"$1,391.55","$1,065,400"
3,Sonoma,Merlot,Red,2000,"6,564.0","1,058.0","7,622.0","31,480.0","$2,019.29","$63,566,200"
4,Sonoma,Meunier,Red,2000,129.0,17.0,146.0,637.0,"$2,069.27","$1,318,700"
...,...,...,...,...,...,...,...,...,...,...
258,Sonoma,Pinot Noir,Red,2016,"13,343.1",385.2,"13,728.3","42,566.3","$3,680.50","$156,665,300"
259,Sonoma,Sangiovese,Red,2016,316.9,0.0,316.9,"1,091.1","$2,314.48","$2,525,300"
260,Sonoma,Syrah-Shiraz,Red,2016,"1,575.6",13.5,"1,589.1","3,618.1","$2,596.01","$9,392,600"
261,Sonoma,TOTAL ALL REDS Includes Other Reds,Red,2016,"39,821.4","1,342.0","41,163.5","130,589.3","$3,008.75","$392,910,600"


In [10]:
red_2015 = sonoma_2001_sorted_copy.query('Year == "2015"')
red_2015[red_2015.duplicated(subset='Varietal', keep=False)]

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
232,Sonoma,Cabernet Franc,Red,2015,569.9,34.3,604.2,"1,357.8","$2,794.59","$3,794,400"
233,Sonoma,Cabernet Franc,Red,2015,569.9,34.3,604.1,"1,357.8","$2,794.59","$3,794,400"
234,Sonoma,Cabernet Sauvignon,Red,2015,"11,808.8","1,014.7","12,823.5","32,947.3","$2,724.17","$89,754,100"
235,Sonoma,Cabernet Sauvignon,Red,2015,"11,808.8","1,014.7","12,823.4","32,947.3","$2,724.17","$89,754,100"
244,Sonoma,Pinot Noir,Red,2015,"12,850.8",750.0,"13,600.8","30,667.2","$3,525.42","$108,114,800"
245,Sonoma,Pinot Noir,Red,2015,"12,850.8",750.0,"13,600.7","30,667.2","$3,525.42","$108,114,800"


In [11]:
red_2015_updated = red_2015.drop_duplicates(subset=['Varietal'], keep='first')
red_2015_updated

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
232,Sonoma,Cabernet Franc,Red,2015,569.9,34.3,604.2,"1,357.8","$2,794.59","$3,794,400"
234,Sonoma,Cabernet Sauvignon,Red,2015,"11,808.8","1,014.7","12,823.5","32,947.3","$2,724.17","$89,754,100"
236,Sonoma,Carignane,Red,2015,162.7,0.0,162.7,305.8,"$2,195.62","$671,500"
237,Sonoma,Grenache,Red,2015,146.4,3.7,150.1,567.4,"$2,901.69","$1,646,500"
238,Sonoma,Malbec,Red,2015,540.5,12.9,553.4,"1,896.3","$2,636.24","$4,999,200"
239,Sonoma,Mataro/Mouvedere,Red,2015,0.0,0.0,0.0,149.7,"$3,082.19","$461,400"
240,Sonoma,Merlot,Red,2015,"4,108.2",43.4,"4,151.6","12,031.4","$1,717.25","$20,661,000"
241,Sonoma,Petit Verdot,Red,2015,266.2,25.7,291.9,953.4,"$2,719.87","$2,593,200"
242,Sonoma,Petite Sirah,Red,2015,645.6,50.4,696.0,"2,347.4","$2,715.79","$6,375,100"
243,Sonoma,Petite Verdot,Red,2015,266.2,25.7,292.0,953.4,"$2,719.87","$2,593,200"


In [12]:
sonoma_2001_fixed2015 = pd.concat([sonoma_2001_sorted_copy.query('Year != "2015"'), red_2015_updated]).sort_values(['Year', 'Varietal']).reset_index(drop=True)
sonoma_2001_fixed2015_copy = sonoma_2001_fixed2015.copy()
sonoma_2001_fixed2015_copy

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,"2,097.0","$2,226.53","$4,668,600"
1,Sonoma,Cabernet Sauvigno,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,417.96","$79,240,400"
2,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,"$1,391.55","$1,065,400"
3,Sonoma,Merlot,Red,2000,"6,564.0","1,058.0","7,622.0","31,480.0","$2,019.29","$63,566,200"
4,Sonoma,Meunier,Red,2000,129.0,17.0,146.0,637.0,"$2,069.27","$1,318,700"
...,...,...,...,...,...,...,...,...,...,...
255,Sonoma,Pinot Noir,Red,2016,"13,343.1",385.2,"13,728.3","42,566.3","$3,680.50","$156,665,300"
256,Sonoma,Sangiovese,Red,2016,316.9,0.0,316.9,"1,091.1","$2,314.48","$2,525,300"
257,Sonoma,Syrah-Shiraz,Red,2016,"1,575.6",13.5,"1,589.1","3,618.1","$2,596.01","$9,392,600"
258,Sonoma,TOTAL ALL REDS Includes Other Reds,Red,2016,"39,821.4","1,342.0","41,163.5","130,589.3","$3,008.75","$392,910,600"


In [13]:
sonoma_2001_fixed2015_copy.query('Year == "2015"')

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
232,Sonoma,Cabernet Franc,Red,2015,569.9,34.3,604.2,"1,357.8","$2,794.59","$3,794,400"
233,Sonoma,Cabernet Sauvignon,Red,2015,"11,808.8","1,014.7","12,823.5","32,947.3","$2,724.17","$89,754,100"
234,Sonoma,Carignane,Red,2015,162.7,0.0,162.7,305.8,"$2,195.62","$671,500"
235,Sonoma,Grenache,Red,2015,146.4,3.7,150.1,567.4,"$2,901.69","$1,646,500"
236,Sonoma,Malbec,Red,2015,540.5,12.9,553.4,"1,896.3","$2,636.24","$4,999,200"
237,Sonoma,Mataro/Mouvedere,Red,2015,0.0,0.0,0.0,149.7,"$3,082.19","$461,400"
238,Sonoma,Merlot,Red,2015,"4,108.2",43.4,"4,151.6","12,031.4","$1,717.25","$20,661,000"
239,Sonoma,Petit Verdot,Red,2015,266.2,25.7,291.9,953.4,"$2,719.87","$2,593,200"
240,Sonoma,Petite Sirah,Red,2015,645.6,50.4,696.0,"2,347.4","$2,715.79","$6,375,100"
241,Sonoma,Petite Verdot,Red,2015,266.2,25.7,292.0,953.4,"$2,719.87","$2,593,200"


In [14]:
# pd.set_option('display.max_rows', 300)

In [15]:
sonoma_2001_fixed2015_copy.query('Year == "2014"')

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
215,Sonoma,Cabernet Franc,Red,2014,571.8,15.8,587.6,"1,913.9","$2,669.35","$5,108,900"
216,Sonoma,Cabernet Sauvignon,Red,2014,"11,689.7","1,078.2","12,767.9","45,772.0","$2,614.37","$119,665,000"
217,Sonoma,Carignane,Red,2014,178.8,1.0,179.8,478.8,"$1,947.37","$932,500"
218,Sonoma,Grenache,Red,2014,119.3,5.2,124.5,701.7,"$2,830.53","$1,986,200"
219,Sonoma,Malbec,Red,2014,491.0,41.8,532.8,"2,688.9","$2,427.49","$6,527,300"
220,Sonoma,Merlot,Red,2014,"4,308.6",27.7,"4,336.2","16,989.6","$1,651.47","$28,057,900"
221,Sonoma,Meunier,Red,2014,88.8,6.6,95.4,607.6,"$2,305.57","$1,400,900"
222,Sonoma,Petite Sirah,Red,2014,630.1,55.1,685.2,"2,873.5","$2,548.12","$7,332,100"
223,Sonoma,Petite Sirah,Red,2014,630.1,55.1,685.2,"2,873.5","$2,548.12","$7,322,100"
224,Sonoma,Petite Verdot,Red,2014,250.0,34.2,284.2,"1,251.1","$2,667.28","$3,337,100"


In [16]:
red_2014 = sonoma_2001_fixed2015_copy.query('Year == "2014"')
red_2014[red_2014.duplicated(subset='Varietal', keep=False)]

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
222,Sonoma,Petite Sirah,Red,2014,630.1,55.1,685.2,"2,873.5","$2,548.12","$7,332,100"
223,Sonoma,Petite Sirah,Red,2014,630.1,55.1,685.2,"2,873.5","$2,548.12","$7,322,100"
225,Sonoma,Pinot Noir,Red,2014,"12,567.9","1,014.4","13,582.7","47,677.2","$3,251.49","$155,022,000"
226,Sonoma,Pinot Noir,Red,2014,"12,567.9","1,014.4","13,582.3","47,677.2","$3,251.49","$155,022,000"
227,Sonoma,Sangiovese,Red,2014,210.2,0.0,210.3,"1,211.9","$2,352.94","$2,851,600"
228,Sonoma,Sangiovese,Red,2014,210.2,0.0,210.2,"1,211.9","$2,352.94","$2,851,600"


In [17]:
red_2014_updated = red_2014.drop_duplicates(subset=['Varietal'], keep='first')
red_2014_updated

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
215,Sonoma,Cabernet Franc,Red,2014,571.8,15.8,587.6,"1,913.9","$2,669.35","$5,108,900"
216,Sonoma,Cabernet Sauvignon,Red,2014,"11,689.7","1,078.2","12,767.9","45,772.0","$2,614.37","$119,665,000"
217,Sonoma,Carignane,Red,2014,178.8,1.0,179.8,478.8,"$1,947.37","$932,500"
218,Sonoma,Grenache,Red,2014,119.3,5.2,124.5,701.7,"$2,830.53","$1,986,200"
219,Sonoma,Malbec,Red,2014,491.0,41.8,532.8,"2,688.9","$2,427.49","$6,527,300"
220,Sonoma,Merlot,Red,2014,"4,308.6",27.7,"4,336.2","16,989.6","$1,651.47","$28,057,900"
221,Sonoma,Meunier,Red,2014,88.8,6.6,95.4,607.6,"$2,305.57","$1,400,900"
222,Sonoma,Petite Sirah,Red,2014,630.1,55.1,685.2,"2,873.5","$2,548.12","$7,332,100"
224,Sonoma,Petite Verdot,Red,2014,250.0,34.2,284.2,"1,251.1","$2,667.28","$3,337,100"
225,Sonoma,Pinot Noir,Red,2014,"12,567.9","1,014.4","13,582.7","47,677.2","$3,251.49","$155,022,000"


In [18]:
sonoma_2001_fixed2014 = pd.concat([sonoma_2001_sorted_copy.query('Year != "2014"'), red_2014_updated]).sort_values(['Year', 'Varietal']).reset_index(drop=True)
sonoma_2001_fixed2014_copy = sonoma_2001_fixed2014.copy()
sonoma_2001_fixed2014_copy

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,"2,097.0","$2,226.53","$4,668,600"
1,Sonoma,Cabernet Sauvigno,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,417.96","$79,240,400"
2,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,"$1,391.55","$1,065,400"
3,Sonoma,Merlot,Red,2000,"6,564.0","1,058.0","7,622.0","31,480.0","$2,019.29","$63,566,200"
4,Sonoma,Meunier,Red,2000,129.0,17.0,146.0,637.0,"$2,069.27","$1,318,700"
...,...,...,...,...,...,...,...,...,...,...
255,Sonoma,Pinot Noir,Red,2016,"13,343.1",385.2,"13,728.3","42,566.3","$3,680.50","$156,665,300"
256,Sonoma,Sangiovese,Red,2016,316.9,0.0,316.9,"1,091.1","$2,314.48","$2,525,300"
257,Sonoma,Syrah-Shiraz,Red,2016,"1,575.6",13.5,"1,589.1","3,618.1","$2,596.01","$9,392,600"
258,Sonoma,TOTAL ALL REDS Includes Other Reds,Red,2016,"39,821.4","1,342.0","41,163.5","130,589.3","$3,008.75","$392,910,600"


## Excel Read in for 2012 and 2013; big dataframe stopped here

In [19]:
df_2012_2013 = pd.read_excel('2013_2012_red_variety.xlsx', sheet_name=['2013', '2012'])
df_2012_2013

{'2013':     Tables_WinegrapeProduction Top 12 Red Varieties__1  \
 0                                                 NaN    
 1                                                 NaN    
 2                                                 NaN    
 3                                                 NaN    
 4                                                 NaN    
 5                                                 NaN    
 6                                                 NaN    
 7                                                 NaN    
 8                                                 NaN    
 9                                                 NaN    
 10                                                NaN    
 11                                                NaN    
 12                                                NaN    
 13                                                NaN    
 14                                                NaN    
 15                                             

In [20]:
df_2013 = df_2012_2013.get('2013')
df_2013_7_columns = df_2013.iloc[:, :7]
df_2013_last_column = df_2013.iloc[:, 8:9]
# df_2013_correct_columns.columns = ['Varietal, Year, Bearing, Non-Bearing, Total, Tons, $/Ton, Total Value']
df_2013_right_columns = pd.concat([df_2013_7_columns, df_2013_last_column], axis=1)
df_2013_right_columns.columns = ['Varietal', 'Year', 'Bearing', 'Non-Bearing', 'Total', 'Tons', '$/Ton', 'Total Value']
df_2013_right_columns

,Varietal,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,NaN,2013.0,604.5,13.2,617.7,"1,917.8","$2,445.23","$4,689,500"
1,NaN,2012.0,661.3,2.7,664.0,"1,906.5","$2,325.46","$4,433,500"
2,NaN,2013.0,"11,830.3","1,070.8","12,901.1","44,255.8","$2,462.99","$109,001,600"
3,NaN,2012.0,"11,929.4",168.0,"12,097.4","47,194.0","$2,313.56","$109,186,200"
4,NaN,2013.0,579.6,4.0,583.6,506.7,"$1,870.18","$947,700"
5,NaN,2012.0,612.4,0.0,612.4,510.6,"$1,879.74","$959,800"
6,NaN,2013.0,425.7,109.4,535.1,"2,706.6","$2,356.19","$6,377,300"
7,NaN,2012.0,332.5,68.3,400.8,"2,469.2","$2,284.02","$5,639,800"
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,2013.0,"4,619.9",27.3,"4,647.2","18,784.5","$1,607.03","$30,187,300"


In [21]:
df_2013_right_columns_droppedna = df_2013_right_columns.dropna(how='all')
df_2013_right_columns_droppedna

,Varietal,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,NaN,2013.0,604.5,13.2,617.7,"1,917.8","$2,445.23","$4,689,500"
1,NaN,2012.0,661.3,2.7,664.0,"1,906.5","$2,325.46","$4,433,500"
2,NaN,2013.0,"11,830.3","1,070.8","12,901.1","44,255.8","$2,462.99","$109,001,600"
3,NaN,2012.0,"11,929.4",168.0,"12,097.4","47,194.0","$2,313.56","$109,186,200"
4,NaN,2013.0,579.6,4.0,583.6,506.7,"$1,870.18","$947,700"
5,NaN,2012.0,612.4,0.0,612.4,510.6,"$1,879.74","$959,800"
6,NaN,2013.0,425.7,109.4,535.1,"2,706.6","$2,356.19","$6,377,300"
7,NaN,2012.0,332.5,68.3,400.8,"2,469.2","$2,284.02","$5,639,800"
9,NaN,2013.0,"4,619.9",27.3,"4,647.2","18,784.5","$1,607.03","$30,187,300"
10,NaN,2012.0,"5,057.1",12.7,"5,069.8","21,627.6","$1,496.57","$32,367,300"


In [22]:
varietal_names = np.array(['Cabernet Franc', 
                            'Cabernet Sauvignon', 
                            'Carignane', 
                            'Malbec', 
                            'Merlot', 
                            'Meunier', 
                            'Petite Sirah', 
                            'Petite Verdot', 
                            'Pinot Noir', 
                            'Sangiovese', 
                            'Syrah-Shiraz', 
                            'Zinfandel',
                            'TOTAL ALL REDS Includes other reds'])
varietal_names_repeated = varietal_names.repeat(2)
varietal_names_repeated

array(['Cabernet Franc', 'Cabernet Franc', 'Cabernet Sauvignon',
       'Cabernet Sauvignon', 'Carignane', 'Carignane', 'Malbec', 'Malbec',
       'Merlot', 'Merlot', 'Meunier', 'Meunier', 'Petite Sirah',
       'Petite Sirah', 'Petite Verdot', 'Petite Verdot', 'Pinot Noir',
       'Pinot Noir', 'Sangiovese', 'Sangiovese', 'Syrah-Shiraz',
       'Syrah-Shiraz', 'Zinfandel', 'Zinfandel',
       'TOTAL ALL REDS Includes other reds',
       'TOTAL ALL REDS Includes other reds'], dtype='<U34')

In [23]:
df_2013_right_columns_droppedna['Varietal'] = varietal_names_repeated
df_2013_right_columns_droppedna['Year'] = df_2013_right_columns_droppedna['Year'].astype('int')
df_2013_right_columns_cleaned = df_2013_right_columns_droppedna.astype(str)
df_2013_right_columns_cleaned

,Varietal,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Cabernet Franc,2013,604.5,13.2,617.7,"1,917.8","$2,445.23","$4,689,500"
1,Cabernet Franc,2012,661.3,2.7,664.0,"1,906.5","$2,325.46","$4,433,500"
2,Cabernet Sauvignon,2013,"11,830.3","1,070.8","12,901.1","44,255.8","$2,462.99","$109,001,600"
3,Cabernet Sauvignon,2012,"11,929.4",168.0,"12,097.4","47,194.0","$2,313.56","$109,186,200"
4,Carignane,2013,579.6,4.0,583.6,506.7,"$1,870.18","$947,700"
5,Carignane,2012,612.4,0.0,612.4,510.6,"$1,879.74","$959,800"
6,Malbec,2013,425.7,109.4,535.1,"2,706.6","$2,356.19","$6,377,300"
7,Malbec,2012,332.5,68.3,400.8,"2,469.2","$2,284.02","$5,639,800"
9,Merlot,2013,"4,619.9",27.3,"4,647.2","18,784.5","$1,607.03","$30,187,300"
10,Merlot,2012,"5,057.1",12.7,"5,069.8","21,627.6","$1,496.57","$32,367,300"


In [24]:
df_2012= df_2012_2013.get('2012')
df_2012 = df_2012.dropna(axis=1, how='all')
df_2012_right_columns = df_2012.iloc[:, :8]
df_2012_right_columns.columns = ['Varietal', 'Year', 'Bearing', 'Non-Bearing', 'Total', 'Tons', '$/Ton', 'Total Value']
df_2012_right_columns['Varietal'] = np.nan
df_2012_right_columns = df_2012_right_columns.dropna(how='all')
df_2012_right_columns

,Varietal,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,NaN,2012.0,662.3,2.4,664.7,"1,906.5","$2,325.46","$4,433,500"
1,NaN,2011.0,655.9,6.7,662.6,"1,568.6","$2,209.15","$3,465,300"
2,NaN,2012.0,"11,904.5",119.4,"12,023.9","47,194.0","$2,313.56","$109,186,200"
3,NaN,2011.0,"11,726.8",291.5,"12,018.3","33,126.8","$2,111.24","$69,938,700"
4,NaN,2012.0,610.4,0.0,610.4,510.6,"$1,879.74","$959,800"
5,NaN,2011.0,615.4,0.0,615.4,373.2,"$2,019.14","$753,600"
6,NaN,2012.0,330.7,68.3,398.9,"2,469.2","$2,284.02","$5,639,800"
8,NaN,2011.0,322.4,76.6,399.0,"1,110.5","$2,340.51","$2,599,200"
9,NaN,2012.0,"5,137.2",1.0,"5,138.2","21,627.6","$1,496.57","$32,367,300"
10,NaN,2011.0,"5,126.6",23.4,"5,149.9","14,670.5","$1,380.85","$20,257,800"


In [25]:
varietal_names = np.array(['Cabernet Franc', 
                            'Cabernet Sauvignon', 
                            'Carignane', 
                            'Malbec', 
                            'Merlot', 
                            'Meunier', 
                            'Petite Sirah', 
                            'Petite Verdot', 
                            'Pinot Noir', 
                            'Sangiovese', 
                            'Syrah-Shiraz', 
                            'Zinfandel',
                            'TOTAL ALL REDS Includes other reds'])
varietal_names_repeated = varietal_names.repeat(2)
varietal_names_repeated

array(['Cabernet Franc', 'Cabernet Franc', 'Cabernet Sauvignon',
       'Cabernet Sauvignon', 'Carignane', 'Carignane', 'Malbec', 'Malbec',
       'Merlot', 'Merlot', 'Meunier', 'Meunier', 'Petite Sirah',
       'Petite Sirah', 'Petite Verdot', 'Petite Verdot', 'Pinot Noir',
       'Pinot Noir', 'Sangiovese', 'Sangiovese', 'Syrah-Shiraz',
       'Syrah-Shiraz', 'Zinfandel', 'Zinfandel',
       'TOTAL ALL REDS Includes other reds',
       'TOTAL ALL REDS Includes other reds'], dtype='<U34')

In [26]:
df_2012_right_columns['Varietal'] = varietal_names_repeated
df_2012_right_columns['Year'] = df_2012_right_columns['Year'].astype('int')
df_2012_right_columns_cleaned = df_2012_right_columns.astype(str)
df_2012_right_columns_cleaned['Total Value'] = df_2012_right_columns_cleaned['Total Value'].str.replace('*', '', regex=False)
df_2012_right_columns_cleaned['Total Value'] = df_2012_right_columns_cleaned['Total Value'].str.replace(' ', '')
df_2012_right_columns_cleaned['$/Ton'] = df_2012_right_columns_cleaned['$/Ton'].str.replace(' ', '')
df_2012_right_columns_cleaned

,Varietal,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Cabernet Franc,2012,662.3,2.4,664.7,"1,906.5","$2,325.46","$4,433,500"
1,Cabernet Franc,2011,655.9,6.7,662.6,"1,568.6","$2,209.15","$3,465,300"
2,Cabernet Sauvignon,2012,"11,904.5",119.4,"12,023.9","47,194.0","$2,313.56","$109,186,200"
3,Cabernet Sauvignon,2011,"11,726.8",291.5,"12,018.3","33,126.8","$2,111.24","$69,938,700"
4,Carignane,2012,610.4,0.0,610.4,510.6,"$1,879.74","$959,800"
5,Carignane,2011,615.4,0.0,615.4,373.2,"$2,019.14","$753,600"
6,Malbec,2012,330.7,68.3,398.9,"2,469.2","$2,284.02","$5,639,800"
8,Malbec,2011,322.4,76.6,399.0,"1,110.5","$2,340.51","$2,599,200"
9,Merlot,2012,"5,137.2",1.0,"5,138.2","21,627.6","$1,496.57","$32,367,300"
10,Merlot,2011,"5,126.6",23.4,"5,149.9","14,670.5","$1,380.85","$20,257,800"


## Show 2012 duplicates with matching versions

In [27]:
report_11_12_13_combined = pd.concat([df_2013_right_columns_cleaned, df_2012_right_columns_cleaned])
report_11_12_13_combined_changed = report_11_12_13_combined.drop_duplicates().sort_values(['Varietal', 'Year']).reset_index(drop=True)
report_11_12_13_combined_changed[report_11_12_13_combined_changed.duplicated(subset=['Varietal','Year'], keep=False)]

,Varietal,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
1,Cabernet Franc,2012,661.3,2.7,664.0,"1,906.5","$2,325.46","$4,433,500"
2,Cabernet Franc,2012,662.3,2.4,664.7,"1,906.5","$2,325.46","$4,433,500"
5,Cabernet Sauvignon,2012,"11,929.4",168.0,"12,097.4","47,194.0","$2,313.56","$109,186,200"
6,Cabernet Sauvignon,2012,"11,904.5",119.4,"12,023.9","47,194.0","$2,313.56","$109,186,200"
9,Carignane,2012,612.4,0.0,612.4,510.6,"$1,879.74","$959,800"
10,Carignane,2012,610.4,0.0,610.4,510.6,"$1,879.74","$959,800"
13,Malbec,2012,332.5,68.3,400.8,"2,469.2","$2,284.02","$5,639,800"
14,Malbec,2012,330.7,68.3,398.9,"2,469.2","$2,284.02","$5,639,800"
17,Merlot,2012,"5,057.1",12.7,"5,069.8","21,627.6","$1,496.57","$32,367,300"
18,Merlot,2012,"5,137.2",1.0,"5,138.2","21,627.6","$1,496.57","$32,367,300"


## Drop 2012 duplicates; compare between 2013's 2012 version and 2012 version

In [28]:
red_11_12_13_drop_duplicates = report_11_12_13_combined_changed.drop_duplicates(subset=['Varietal', 'Year'], keep='first')
red_11_12_13_drop_duplicates

,Varietal,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Cabernet Franc,2011,655.9,6.7,662.6,"1,568.6","$2,209.15","$3,465,300"
1,Cabernet Franc,2012,661.3,2.7,664.0,"1,906.5","$2,325.46","$4,433,500"
3,Cabernet Franc,2013,604.5,13.2,617.7,"1,917.8","$2,445.23","$4,689,500"
4,Cabernet Sauvignon,2011,"11,726.8",291.5,"12,018.3","33,126.8","$2,111.24","$69,938,700"
5,Cabernet Sauvignon,2012,"11,929.4",168.0,"12,097.4","47,194.0","$2,313.56","$109,186,200"
7,Cabernet Sauvignon,2013,"11,830.3","1,070.8","12,901.1","44,255.8","$2,462.99","$109,001,600"
8,Carignane,2011,615.4,0.0,615.4,373.2,"$2,019.14","$753,600"
9,Carignane,2012,612.4,0.0,612.4,510.6,"$1,879.74","$959,800"
11,Carignane,2013,579.6,4.0,583.6,506.7,"$1,870.18","$947,700"
12,Malbec,2011,322.4,76.6,399.0,"1,110.5","$2,340.51","$2,599,200"


In [29]:
red_11_12_13_drop_duplicates['County'] = 'Sonoma'
red_11_12_13_drop_duplicates['Grape Color'] = 'Red'
county_column = red_11_12_13_drop_duplicates['County']
grape_color_column = red_11_12_13_drop_duplicates['Grape Color']
red_11_12_13_drop_duplicates = red_11_12_13_drop_duplicates.drop(['County', 'Grape Color'], axis=1)
red_11_12_13_drop_duplicates.insert(0, 'County', county_column)
red_11_12_13_drop_duplicates.insert(2, 'Grape Color', grape_color_column)
red_11_12_13_drop_duplicates

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2011,655.9,6.7,662.6,"1,568.6","$2,209.15","$3,465,300"
1,Sonoma,Cabernet Franc,Red,2012,661.3,2.7,664.0,"1,906.5","$2,325.46","$4,433,500"
3,Sonoma,Cabernet Franc,Red,2013,604.5,13.2,617.7,"1,917.8","$2,445.23","$4,689,500"
4,Sonoma,Cabernet Sauvignon,Red,2011,"11,726.8",291.5,"12,018.3","33,126.8","$2,111.24","$69,938,700"
5,Sonoma,Cabernet Sauvignon,Red,2012,"11,929.4",168.0,"12,097.4","47,194.0","$2,313.56","$109,186,200"
7,Sonoma,Cabernet Sauvignon,Red,2013,"11,830.3","1,070.8","12,901.1","44,255.8","$2,462.99","$109,001,600"
8,Sonoma,Carignane,Red,2011,615.4,0.0,615.4,373.2,"$2,019.14","$753,600"
9,Sonoma,Carignane,Red,2012,612.4,0.0,612.4,510.6,"$1,879.74","$959,800"
11,Sonoma,Carignane,Red,2013,579.6,4.0,583.6,506.7,"$1,870.18","$947,700"
12,Sonoma,Malbec,Red,2011,322.4,76.6,399.0,"1,110.5","$2,340.51","$2,599,200"


### Now because only 2012 is matched with 2013's 2012 versiona and 2012 version; we can only revised 2012 from the big dataframe

In [30]:
sonoma_2012 = red_11_12_13_drop_duplicates.query('Year == "2012"')
sonoma_2012

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
1,Sonoma,Cabernet Franc,Red,2012,661.3,2.7,664.0,"1,906.5","$2,325.46","$4,433,500"
5,Sonoma,Cabernet Sauvignon,Red,2012,"11,929.4",168.0,"12,097.4","47,194.0","$2,313.56","$109,186,200"
9,Sonoma,Carignane,Red,2012,612.4,0.0,612.4,510.6,"$1,879.74","$959,800"
13,Sonoma,Malbec,Red,2012,332.5,68.3,400.8,"2,469.2","$2,284.02","$5,639,800"
17,Sonoma,Merlot,Red,2012,"5,057.1",12.7,"5,069.8","21,627.6","$1,496.57","$32,367,300"
21,Sonoma,Meunier,Red,2012,110.0,15.0,125.0,581.2,"$2,735.16","$1,589,700"
25,Sonoma,Petite Sirah,Red,2012,573.2,16.7,589.9,"3,149.1","$2,424.27","$7,634,300"
29,Sonoma,Petite Verdot,Red,2012,210.9,5.6,216.5,"1,162.6","$2,669.78","$3,103,900"
33,Sonoma,Pinot Noir,Red,2012,"11,753.1",375.2,"12,128.3","52,793.2","$3,014.62","$159,151,500"
37,Sonoma,Sangiovese,Red,2012,91.2,66.2,157.5,"1,341.5","$2,270.04","$3,045,300"


In [31]:
sonoma_2001_fixed2014_added2012 = pd.concat([sonoma_2001_fixed2014, sonoma_2012]).sort_values(['Year', 'Varietal']).reset_index(drop=True)
sonoma_2001_fixed2014_added2012

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,"2,097.0","$2,226.53","$4,668,600"
1,Sonoma,Cabernet Sauvigno,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,417.96","$79,240,400"
2,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,"$1,391.55","$1,065,400"
3,Sonoma,Merlot,Red,2000,"6,564.0","1,058.0","7,622.0","31,480.0","$2,019.29","$63,566,200"
4,Sonoma,Meunier,Red,2000,129.0,17.0,146.0,637.0,"$2,069.27","$1,318,700"
...,...,...,...,...,...,...,...,...,...,...
268,Sonoma,Pinot Noir,Red,2016,"13,343.1",385.2,"13,728.3","42,566.3","$3,680.50","$156,665,300"
269,Sonoma,Sangiovese,Red,2016,316.9,0.0,316.9,"1,091.1","$2,314.48","$2,525,300"
270,Sonoma,Syrah-Shiraz,Red,2016,"1,575.6",13.5,"1,589.1","3,618.1","$2,596.01","$9,392,600"
271,Sonoma,TOTAL ALL REDS Includes Other Reds,Red,2016,"39,821.4","1,342.0","41,163.5","130,589.3","$3,008.75","$392,910,600"


## Matching 2011 and 2013

In [32]:
sonoma_2011 = red_11_12_13_drop_duplicates.query('Year == "2011"')
sonoma_2013 = red_11_12_13_drop_duplicates.query('Year == "2013"')

In [33]:
sonoma_2011 = sonoma_2011.replace('TOTAL ALL REDS Includes other reds', 'TOTAL ALL REDS Includes Other Reds')
sonoma_2011

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2011,655.9,6.7,662.6,"1,568.6","$2,209.15","$3,465,300"
4,Sonoma,Cabernet Sauvignon,Red,2011,"11,726.8",291.5,"12,018.3","33,126.8","$2,111.24","$69,938,700"
8,Sonoma,Carignane,Red,2011,615.4,0.0,615.4,373.2,"$2,019.14","$753,600"
12,Sonoma,Malbec,Red,2011,322.4,76.6,399.0,"1,110.5","$2,340.51","$2,599,200"
16,Sonoma,Merlot,Red,2011,"5,126.6",23.4,"5,149.9","14,670.5","$1,380.85","$20,257,800"
20,Sonoma,Meunier,Red,2011,108.5,15.0,123.5,301.4,"$2,698.80","$813,500"
24,Sonoma,Petite Sirah,Red,2011,575.0,22.2,597.1,"2,521.1","$2,325.25","$5,862,200"
28,Sonoma,Petite Verdot,Red,2011,209.2,2.6,211.8,851.7,"$2,461.02","$2,096,100"
32,Sonoma,Pinot Noir,Red,2011,"11,862.6",377.1,"12,239.7","28,350.2","$2,902.47","$82,285,700"
36,Sonoma,Sangiovese,Red,2011,89.5,36.5,126.0,845.2,"$2,204.51","$1,863,300"


In [34]:
sonoma_2013 = sonoma_2013.replace('TOTAL ALL REDS Includes other reds', 'TOTAL ALL REDS Includes Other Reds')
sonoma_2013

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
3,Sonoma,Cabernet Franc,Red,2013,604.5,13.2,617.7,"1,917.8","$2,445.23","$4,689,500"
7,Sonoma,Cabernet Sauvignon,Red,2013,"11,830.3","1,070.8","12,901.1","44,255.8","$2,462.99","$109,001,600"
11,Sonoma,Carignane,Red,2013,579.6,4.0,583.6,506.7,"$1,870.18","$947,700"
15,Sonoma,Malbec,Red,2013,425.7,109.4,535.1,"2,706.6","$2,356.19","$6,377,300"
19,Sonoma,Merlot,Red,2013,"4,619.9",27.3,"4,647.2","18,784.5","$1,607.03","$30,187,300"
23,Sonoma,Meunier,Red,2013,91.8,6.6,98.4,716.9,"$2,285.40","$1,638,500"
27,Sonoma,Petite Sirah,Red,2013,598.3,83.5,681.8,"2,856.8","$2,477.06","$7,076,500"
31,Sonoma,Petite Verdot,Red,2013,236.9,30.5,267.4,"1,120.0","$2,840.39","$3,181,300"
35,Sonoma,Pinot Noir,Red,2013,"12,539.3",948.1,"13,487.4","54,826.8","$3,079.99","$168,866,000"
39,Sonoma,Sangiovese,Red,2013,195.9,12.7,208.6,"1,141.1","$2,383.10","$2,719,400"


In [35]:
sonoma_2001_fixed2014_added2012.query('Year == "2011"')

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
189,Sonoma,Cabernet Franc,Red,2011,655.9,6.7,662.6,"1,568.6","$2,209.15","$3,465,300"
190,Sonoma,Cabernet Sauvignon,Red,2011,"11,726.8",291.5,"12,018.3","33,126.8","$2,111.24","$69,938,700"
191,Sonoma,Carignane,Red,2011,615.4,0.0,615.4,373.2,"$2,019.14","$753,600"
192,Sonoma,Malbec,Red,2011,322.4,76.6,399.0,"1,110.5","$2,340.51","$2,599,200"
193,Sonoma,Merlot,Red,2011,"5,126.6",23.4,"5,149.9","14,670.5","$1,380.85","$20,257,800"
194,Sonoma,Meunier,Red,2011,108.5,15.0,123.5,301.4,"$2,698.80","$813,500"
195,Sonoma,Petite Sirah,Red,2011,575.0,22.2,597.1,"2,521.1","$2,325.25","$5,862,200"
196,Sonoma,Petite Verdot,Red,2011,209.2,2.6,211.8,851.7,"$2,461.02","$2,096,100"
197,Sonoma,Pinot Noir,Red,2011,"11,862.6",377.1,"12,239.7","28,350.2","$2,902.47","$82,285,700"
198,Sonoma,Sangiovese,Red,2011,89.5,36.5,126.0,845.2,"$2,204.51","$1,863,300"


In [36]:
sonoma_2011_matched = pd.concat([sonoma_2001_fixed2014_added2012.query('Year == "2011"'), sonoma_2011]).sort_values('Varietal').drop_duplicates(subset=['Varietal'], keep='first')
sonoma_2011_matched

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
189,Sonoma,Cabernet Franc,Red,2011,655.9,6.7,662.6,"1,568.6","$2,209.15","$3,465,300"
190,Sonoma,Cabernet Sauvignon,Red,2011,"11,726.8",291.5,"12,018.3","33,126.8","$2,111.24","$69,938,700"
191,Sonoma,Carignane,Red,2011,615.4,0.0,615.4,373.2,"$2,019.14","$753,600"
192,Sonoma,Malbec,Red,2011,322.4,76.6,399.0,"1,110.5","$2,340.51","$2,599,200"
193,Sonoma,Merlot,Red,2011,"5,126.6",23.4,"5,149.9","14,670.5","$1,380.85","$20,257,800"
194,Sonoma,Meunier,Red,2011,108.5,15.0,123.5,301.4,"$2,698.80","$813,500"
24,Sonoma,Petite Sirah,Red,2011,575.0,22.2,597.1,"2,521.1","$2,325.25","$5,862,200"
196,Sonoma,Petite Verdot,Red,2011,209.2,2.6,211.8,851.7,"$2,461.02","$2,096,100"
32,Sonoma,Pinot Noir,Red,2011,"11,862.6",377.1,"12,239.7","28,350.2","$2,902.47","$82,285,700"
36,Sonoma,Sangiovese,Red,2011,89.5,36.5,126.0,845.2,"$2,204.51","$1,863,300"


In [37]:
sonoma_2001_fixed2014_added2011_12 = pd.concat([sonoma_2001_fixed2014_added2012.query('Year != "2011"'), sonoma_2011]).sort_values(['Year', 'Varietal']).reset_index(drop=True)
sonoma_2001_fixed2014_added2011_12

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,"2,097.0","$2,226.53","$4,668,600"
1,Sonoma,Cabernet Sauvigno,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,417.96","$79,240,400"
2,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,"$1,391.55","$1,065,400"
3,Sonoma,Merlot,Red,2000,"6,564.0","1,058.0","7,622.0","31,480.0","$2,019.29","$63,566,200"
4,Sonoma,Meunier,Red,2000,129.0,17.0,146.0,637.0,"$2,069.27","$1,318,700"
...,...,...,...,...,...,...,...,...,...,...
268,Sonoma,Pinot Noir,Red,2016,"13,343.1",385.2,"13,728.3","42,566.3","$3,680.50","$156,665,300"
269,Sonoma,Sangiovese,Red,2016,316.9,0.0,316.9,"1,091.1","$2,314.48","$2,525,300"
270,Sonoma,Syrah-Shiraz,Red,2016,"1,575.6",13.5,"1,589.1","3,618.1","$2,596.01","$9,392,600"
271,Sonoma,TOTAL ALL REDS Includes Other Reds,Red,2016,"39,821.4","1,342.0","41,163.5","130,589.3","$3,008.75","$392,910,600"


In [38]:
sonoma_2013_matched = pd.concat([sonoma_2001_fixed2014_added2011_12.query('Year == "2013"'), sonoma_2013]).sort_values('Varietal').drop_duplicates()
sonoma_2013_matched

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
215,Sonoma,Cabernet Franc,Red,2013,604.5,13.2,617.7,"1,917.8","$2,445.23","$4,689,500"
216,Sonoma,Cabernet Sauvignon,Red,2013,"11,830.3","1,070.8","12,901.1","44,255.8","$2,462.99","$109,001,600"
217,Sonoma,Carignane,Red,2013,579.6,4.0,583.6,506.7,"$1,870.18","$947,700"
218,Sonoma,Malbec,Red,2013,425.7,109.4,535.1,"2,706.6","$2,356.19","$6,377,300"
219,Sonoma,Merlot,Red,2013,"4,619.9",27.3,"4,647.2","18,784.5","$1,607.03","$30,187,300"
220,Sonoma,Meunier,Red,2013,91.8,6.6,98.4,716.9,"$2,285.40","$1,638,500"
27,Sonoma,Petite Sirah,Red,2013,598.3,83.5,681.8,"2,856.8","$2,477.06","$7,076,500"
222,Sonoma,Petite Verdot,Red,2013,236.9,30.5,267.4,"1,120.0","$2,840.39","$3,181,300"
35,Sonoma,Pinot Noir,Red,2013,"12,539.3",948.1,"13,487.4","54,826.8","$3,079.99","$168,866,000"
39,Sonoma,Sangiovese,Red,2013,195.9,12.7,208.6,"1,141.1","$2,383.10","$2,719,400"


In [39]:
sonoma_2001_fixed11_16 = pd.concat([sonoma_2001_fixed2014_added2011_12.query('Year != "2013"'), sonoma_2013_matched]).sort_values(['Year', 'Varietal']).reset_index(drop=True)
sonoma_2001_fixed11_16

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,"2,097.0","$2,226.53","$4,668,600"
1,Sonoma,Cabernet Sauvigno,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,417.96","$79,240,400"
2,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,"$1,391.55","$1,065,400"
3,Sonoma,Merlot,Red,2000,"6,564.0","1,058.0","7,622.0","31,480.0","$2,019.29","$63,566,200"
4,Sonoma,Meunier,Red,2000,129.0,17.0,146.0,637.0,"$2,069.27","$1,318,700"
...,...,...,...,...,...,...,...,...,...,...
268,Sonoma,Pinot Noir,Red,2016,"13,343.1",385.2,"13,728.3","42,566.3","$3,680.50","$156,665,300"
269,Sonoma,Sangiovese,Red,2016,316.9,0.0,316.9,"1,091.1","$2,314.48","$2,525,300"
270,Sonoma,Syrah-Shiraz,Red,2016,"1,575.6",13.5,"1,589.1","3,618.1","$2,596.01","$9,392,600"
271,Sonoma,TOTAL ALL REDS Includes Other Reds,Red,2016,"39,821.4","1,342.0","41,163.5","130,589.3","$3,008.75","$392,910,600"


In [40]:
sonoma_2001_fixed11_16['Varietal'].unique()

array(['Cabernet Franc', 'Cabernet Sauvigno', 'Carignane', 'Merlot',
       'Meunier', 'Napa Gamay', 'Other Reds', 'Petite Sirah',
       'Petite Verdot', 'Pinot Noir', 'Sangiovese', 'Syrah-Shiraz',
       'TOTAL ALL REDS NOT including Other Reds', 'Zinfandel', 'Malbec',
       'Cabernet Sauvignon', 'TOTAL ALL REDS Includes Other Reds',
       'TOTAL ALL REDS Includes other reds', 'Grenache',
       'Mataro/Mouvedere', 'Petit Verdot'], dtype=object)

In [41]:
sonoma_2001_fixed11_16['Varietal'] = sonoma_2001_fixed11_16['Varietal'].replace('TOTAL ALL REDS Includes other reds', 'TOTAL ALL REDS Includes Other Reds')
sonoma_2001_fixed11_16['Varietal'] = sonoma_2001_fixed11_16['Varietal'].replace('TOTAL ALL REDS NOT including Other Reds', 'TOTAL ALL REDS NOT Including Other Reds')
sonoma_2001_fixed11_16['Varietal'].unique()

array(['Cabernet Franc', 'Cabernet Sauvigno', 'Carignane', 'Merlot',
       'Meunier', 'Napa Gamay', 'Other Reds', 'Petite Sirah',
       'Petite Verdot', 'Pinot Noir', 'Sangiovese', 'Syrah-Shiraz',
       'TOTAL ALL REDS NOT Including Other Reds', 'Zinfandel', 'Malbec',
       'Cabernet Sauvignon', 'TOTAL ALL REDS Includes Other Reds',
       'Grenache', 'Mataro/Mouvedere', 'Petit Verdot'], dtype=object)

In [42]:
sonoma_2001_fixed11_16

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,"2,097.0","$2,226.53","$4,668,600"
1,Sonoma,Cabernet Sauvigno,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,417.96","$79,240,400"
2,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,"$1,391.55","$1,065,400"
3,Sonoma,Merlot,Red,2000,"6,564.0","1,058.0","7,622.0","31,480.0","$2,019.29","$63,566,200"
4,Sonoma,Meunier,Red,2000,129.0,17.0,146.0,637.0,"$2,069.27","$1,318,700"
...,...,...,...,...,...,...,...,...,...,...
268,Sonoma,Pinot Noir,Red,2016,"13,343.1",385.2,"13,728.3","42,566.3","$3,680.50","$156,665,300"
269,Sonoma,Sangiovese,Red,2016,316.9,0.0,316.9,"1,091.1","$2,314.48","$2,525,300"
270,Sonoma,Syrah-Shiraz,Red,2016,"1,575.6",13.5,"1,589.1","3,618.1","$2,596.01","$9,392,600"
271,Sonoma,TOTAL ALL REDS Includes Other Reds,Red,2016,"39,821.4","1,342.0","41,163.5","130,589.3","$3,008.75","$392,910,600"


## Still need to fix 2010, 2009, 2008, 2007 (need to combine)

### 2006 discrepancy

In [43]:
red_2006 = sonoma_2001_fixed11_16.query('Year == "2006"').sort_values('Varietal')


red_2006[red_2006.duplicated(subset='Varietal', keep=False)]

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
116,Sonoma,Cabernet Franc,Red,2006,679.1,59.2,738.9,"1,664.9","$2,395.21","$3,987,700"
117,Sonoma,Cabernet Franc,Red,2006,679.7,59.2,738.9,"1,664.9","$2,395.21","$3,987,800"


In [44]:
red_2006_updated = red_2006.drop_duplicates(subset=['Varietal'], keep='first')
red_2006_updated

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
116,Sonoma,Cabernet Franc,Red,2006,679.1,59.2,738.9,"1,664.9","$2,395.21","$3,987,700"
118,Sonoma,Cabernet Sauvignon,Red,2006,"11,527.2",547.4,"12,074.6","38,410.3","$2,265.69","$87,025,900"
119,Sonoma,Carignane,Red,2006,188.0,1.6,189.6,394.4,"$1,804.52","$711,800"
120,Sonoma,Malbec,Red,2006,241.1,43.3,284.4,"1,393.0","$2,483.61","$3,459,700"
121,Sonoma,Merlot,Red,2006,"6,697.4",197.5,"6,894.9","26,015.2","$1,529.52","$39,790,800"
122,Sonoma,Meunier,Red,2006,103.0,3.0,106.0,682.8,"$2,166.97","$1,479,700"
123,Sonoma,Other Reds,Red,2006,517.5,37.3,554.8,"1,469.6","$2,110.57","$3,101,700"
124,Sonoma,Petite Sirah,Red,2006,417.3,65.7,483.0,"1,708.7","$2,371.27","$4,051,800"
125,Sonoma,Petite Verdot,Red,2006,167.9,44.7,212.6,651.2,"$2,677.21","$1,743,400"
126,Sonoma,Pinot Noir,Red,2006,"9,884.7","1,146.2","11,030.9","38,453.8","$2,507.31","$96,415,600"


## Fixed 2006 discrepancy

In [45]:
sonoma_2001_fixed11_16_06 = pd.concat([sonoma_2001_fixed11_16.query('Year != "2006"'), red_2006_updated]).sort_values(['Year', 'Varietal']).reset_index(drop=True)
sonoma_2001_fixed11_16_06

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,"2,097.0","$2,226.53","$4,668,600"
1,Sonoma,Cabernet Sauvigno,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,417.96","$79,240,400"
2,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,"$1,391.55","$1,065,400"
3,Sonoma,Merlot,Red,2000,"6,564.0","1,058.0","7,622.0","31,480.0","$2,019.29","$63,566,200"
4,Sonoma,Meunier,Red,2000,129.0,17.0,146.0,637.0,"$2,069.27","$1,318,700"
...,...,...,...,...,...,...,...,...,...,...
267,Sonoma,Pinot Noir,Red,2016,"13,343.1",385.2,"13,728.3","42,566.3","$3,680.50","$156,665,300"
268,Sonoma,Sangiovese,Red,2016,316.9,0.0,316.9,"1,091.1","$2,314.48","$2,525,300"
269,Sonoma,Syrah-Shiraz,Red,2016,"1,575.6",13.5,"1,589.1","3,618.1","$2,596.01","$9,392,600"
270,Sonoma,TOTAL ALL REDS Includes Other Reds,Red,2016,"39,821.4","1,342.0","41,163.5","130,589.3","$3,008.75","$392,910,600"


In [46]:
red_2005_updated = sonoma_2001_fixed11_16_06.query('Year == "2005"').drop_duplicates(subset=['Varietal'], keep='first')
sonoma_2001_fixed11_16_06_05 = pd.concat([sonoma_2001_fixed11_16_06.query('Year != "2005"'), red_2005_updated]).sort_values(['Year', 'Varietal']).reset_index(drop=True)
sonoma_2001_fixed11_16_06_05

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,"2,097.0","$2,226.53","$4,668,600"
1,Sonoma,Cabernet Sauvigno,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,417.96","$79,240,400"
2,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,"$1,391.55","$1,065,400"
3,Sonoma,Merlot,Red,2000,"6,564.0","1,058.0","7,622.0","31,480.0","$2,019.29","$63,566,200"
4,Sonoma,Meunier,Red,2000,129.0,17.0,146.0,637.0,"$2,069.27","$1,318,700"
...,...,...,...,...,...,...,...,...,...,...
254,Sonoma,Pinot Noir,Red,2016,"13,343.1",385.2,"13,728.3","42,566.3","$3,680.50","$156,665,300"
255,Sonoma,Sangiovese,Red,2016,316.9,0.0,316.9,"1,091.1","$2,314.48","$2,525,300"
256,Sonoma,Syrah-Shiraz,Red,2016,"1,575.6",13.5,"1,589.1","3,618.1","$2,596.01","$9,392,600"
257,Sonoma,TOTAL ALL REDS Includes Other Reds,Red,2016,"39,821.4","1,342.0","41,163.5","130,589.3","$3,008.75","$392,910,600"


## Skipped 2004 because there was no problem found 

In [47]:
sonoma_2001_fixed11_16_06_05_04 = sonoma_2001_fixed11_16_06_05


red_2003_updated = sonoma_2001_fixed11_16_06_05_04.query('Year == "2003"').drop_duplicates(subset=['Varietal'], keep='first')
sonoma_2001_fixed11_16_06_05_04_03 = pd.concat([sonoma_2001_fixed11_16_06_05_04.query('Year != "2003"'), red_2003_updated]).sort_values(['Year', 'Varietal']).reset_index(drop=True)
sonoma_2001_fixed11_16_06_05_04_03

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,"2,097.0","$2,226.53","$4,668,600"
1,Sonoma,Cabernet Sauvigno,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,417.96","$79,240,400"
2,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,"$1,391.55","$1,065,400"
3,Sonoma,Merlot,Red,2000,"6,564.0","1,058.0","7,622.0","31,480.0","$2,019.29","$63,566,200"
4,Sonoma,Meunier,Red,2000,129.0,17.0,146.0,637.0,"$2,069.27","$1,318,700"
...,...,...,...,...,...,...,...,...,...,...
250,Sonoma,Pinot Noir,Red,2016,"13,343.1",385.2,"13,728.3","42,566.3","$3,680.50","$156,665,300"
251,Sonoma,Sangiovese,Red,2016,316.9,0.0,316.9,"1,091.1","$2,314.48","$2,525,300"
252,Sonoma,Syrah-Shiraz,Red,2016,"1,575.6",13.5,"1,589.1","3,618.1","$2,596.01","$9,392,600"
253,Sonoma,TOTAL ALL REDS Includes Other Reds,Red,2016,"39,821.4","1,342.0","41,163.5","130,589.3","$3,008.75","$392,910,600"


In [48]:
red_2002_updated = sonoma_2001_fixed11_16_06_05_04_03.query('Year == "2002"').drop_duplicates(subset=['Varietal'], keep='first')
sonoma_2001_fixed11_16_06_05_04_03_02 = pd.concat([sonoma_2001_fixed11_16_06_05_04_03.query('Year != "2002"'), red_2002_updated]).sort_values(['Year', 'Varietal']).reset_index(drop=True)
sonoma_2001_fixed11_16_06_05_04_03_02_01 = sonoma_2001_fixed11_16_06_05_04_03_02
sonoma_2001_fixed11_16_06_05_04_03_02_01

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,"2,097.0","$2,226.53","$4,668,600"
1,Sonoma,Cabernet Sauvigno,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,417.96","$79,240,400"
2,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,"$1,391.55","$1,065,400"
3,Sonoma,Merlot,Red,2000,"6,564.0","1,058.0","7,622.0","31,480.0","$2,019.29","$63,566,200"
4,Sonoma,Meunier,Red,2000,129.0,17.0,146.0,637.0,"$2,069.27","$1,318,700"
...,...,...,...,...,...,...,...,...,...,...
237,Sonoma,Pinot Noir,Red,2016,"13,343.1",385.2,"13,728.3","42,566.3","$3,680.50","$156,665,300"
238,Sonoma,Sangiovese,Red,2016,316.9,0.0,316.9,"1,091.1","$2,314.48","$2,525,300"
239,Sonoma,Syrah-Shiraz,Red,2016,"1,575.6",13.5,"1,589.1","3,618.1","$2,596.01","$9,392,600"
240,Sonoma,TOTAL ALL REDS Includes Other Reds,Red,2016,"39,821.4","1,342.0","41,163.5","130,589.3","$3,008.75","$392,910,600"


## Read in 2000 crop report

In [49]:
df_2000_1999 = pd.read_excel('2020 crop report.xlsx')
df_2000_1999 = df_2000_1999.drop(df_2000_1999.index[-2:])
df_2000_1999

,Tables VARIETY YEAR__1,Tables BEARING__2,Tables NON BEARING__3,Tables __4,Tables __5,Tables __6,Tables TONS__7,Tables PER__8,Tables __9,Tables TOTAL VALUE__10,...,type,uploaded_from,doc_meta_data,folder_name,folder_id,created_at_iso,modified_at_iso,status,review_url,user_doc_id
0,Cabernet Franc,523,NaN,151,674,NaN,"2,097","$ 2,227",NaN,"4,668,600",...,financial_statement__DFSGu__CFuBF,web,NaN,NaN,NaN,2023-05-19T00:52:53+00:00,2023-05-19T00:53:37+00:00,reviewing,https://app.docsumo.com/review-document/8e83ce...,NaN
1,NaN,578,NaN,101,679,NaN,"1,769","$ 2,047",NaN,"$ 3,622,400",...,financial_statement__DFSGu__CFuBF,web,NaN,NaN,NaN,2023-05-19T00:52:53+00:00,2023-05-19T00:53:37+00:00,reviewing,https://app.docsumo.com/review-document/8e83ce...,NaN
2,Cabernet Sauv,"7,328",NaN,"3,688","11,016",NaN,"32,772","$ 2,418",NaN,"$ 79,240,400",...,financial_statement__DFSGu__CFuBF,web,NaN,NaN,NaN,2023-05-19T00:52:53+00:00,2023-05-19T00:53:37+00:00,reviewing,https://app.docsumo.com/review-document/8e83ce...,NaN
3,NaN,"7,625",NaN,"2,298","9,923",NaN,"21,037","$ 2,158",NaN,"$ 45,400,600",...,financial_statement__DFSGu__CFuBF,web,NaN,NaN,NaN,2023-05-19T00:52:53+00:00,2023-05-19T00:53:37+00:00,reviewing,https://app.docsumo.com/review-document/8e83ce...,NaN
4,Carignane,196,NaN,NaN,197,NaN,766,"1,392",NaN,"$ 1,065,400",...,financial_statement__DFSGu__CFuBF,web,NaN,NaN,NaN,2023-05-19T00:52:53+00:00,2023-05-19T00:53:37+00:00,reviewing,https://app.docsumo.com/review-document/8e83ce...,NaN
5,NaN,201,NaN,1,202,NaN,590,"$ 1,215",NaN,"$ 717,200",...,financial_statement__DFSGu__CFuBF,web,NaN,NaN,NaN,2023-05-19T00:52:53+00:00,2023-05-19T00:53:37+00:00,reviewing,https://app.docsumo.com/review-document/8e83ce...,NaN
6,Merlot,"6,564",NaN,"1,058","7,622",NaN,"31,480","$ 2,019",NaN,"63,566,200",...,financial_statement__DFSGu__CFuBF,web,NaN,NaN,NaN,2023-05-19T00:52:53+00:00,2023-05-19T00:53:37+00:00,reviewing,https://app.docsumo.com/review-document/8e83ce...,NaN
7,NaN,"6,276",NaN,848,"7,124",NaN,"24,558","$ 1,979",NaN,"48,599,700",...,financial_statement__DFSGu__CFuBF,web,NaN,NaN,NaN,2023-05-19T00:52:53+00:00,2023-05-19T00:53:37+00:00,reviewing,https://app.docsumo.com/review-document/8e83ce...,NaN
8,Meunier,129,NaN,17,146,NaN,637,"$ 2,069",NaN,"1,318,700",...,financial_statement__DFSGu__CFuBF,web,NaN,NaN,NaN,2023-05-19T00:52:53+00:00,2023-05-19T00:53:37+00:00,reviewing,https://app.docsumo.com/review-document/8e83ce...,NaN
9,NaN,129,NaN,17,146,NaN,426,"$ 1,817",NaN,"774,807",...,financial_statement__DFSGu__CFuBF,web,NaN,NaN,NaN,2023-05-19T00:52:53+00:00,2023-05-19T00:53:37+00:00,reviewing,https://app.docsumo.com/review-document/8e83ce...,NaN


In [50]:
correct_col_1999_2000 = df_2000_1999.iloc[:, :10].dropna(axis=1, how='all')
correct_col_1999_2000.insert(1, 'Year', [2000, 1999] * (len(correct_col_1999_2000) // 2) + [2000] * (len(correct_col_1999_2000) % 2))
correct_col_1999_2000.columns = ['Varietal', 'Year', 'Bearing', 'Non-Bearing', 'Total', 'Tons', '$/Ton', 'Total Value']
correct_col_1999_2000
varietal_names = np.array(['Cabernet Franc', 
                            'Cabernet Sauvignon', 
                            'Carignane', 
                            'Merlot', 
                            'Meunier', 
                            'Napa Gamay',
                            'Petite Sirah', 
                            'Petite Verdot', 
                            'Pinot Noir', 
                            'Sangiovese', 
                            'Syrah-Shiraz', 
                            'Zinfandel',
                            'Other Reds',
                            'TOTAL ALL REDS NOT Including Other Reds'])
varietal_names_repeated = varietal_names.repeat(2)
varietal_names_repeated
correct_col_1999_2000['Varietal'] = varietal_names_repeated
correct_col_1999_2000

,Varietal,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Cabernet Franc,2000,523,151,674,"2,097","$ 2,227","4,668,600"
1,Cabernet Franc,1999,578,101,679,"1,769","$ 2,047","$ 3,622,400"
2,Cabernet Sauvignon,2000,"7,328","3,688","11,016","32,772","$ 2,418","$ 79,240,400"
3,Cabernet Sauvignon,1999,"7,625","2,298","9,923","21,037","$ 2,158","$ 45,400,600"
4,Carignane,2000,196,NaN,197,766,"1,392","$ 1,065,400"
5,Carignane,1999,201,1,202,590,"$ 1,215","$ 717,200"
6,Merlot,2000,"6,564","1,058","7,622","31,480","$ 2,019","63,566,200"
7,Merlot,1999,"6,276",848,"7,124","24,558","$ 1,979","48,599,700"
8,Meunier,2000,129,17,146,637,"$ 2,069","1,318,700"
9,Meunier,1999,129,17,146,426,"$ 1,817","774,807"


In [51]:
correct_col_1999_2000 = correct_col_1999_2000.fillna('1')
correct_col_1999_2000['Bearing'] = correct_col_1999_2000['Bearing'] + '.0'
correct_col_1999_2000['Non-Bearing'] = correct_col_1999_2000['Non-Bearing'] + '.0'
correct_col_1999_2000['Total'] = correct_col_1999_2000['Total'] + '.0'
correct_col_1999_2000['Tons'] = correct_col_1999_2000['Tons'] + '.0'
correct_col_1999_2000['$/Ton'] = '$'+ correct_col_1999_2000['$/Ton'].str.replace(' ', '').str.replace('[$S]', '', regex=True) + '.0'
correct_col_1999_2000['Total Value'] = '$'+ correct_col_1999_2000['Total Value'].str.replace(' ', '').str.replace('[$S]', '', regex=True)
correct_col_1999_2000 = correct_col_1999_2000.astype(str)
correct_col_1999_2000

,Varietal,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Cabernet Franc,2000,523.0,151.0,674.0,"2,097.0","$2,227.0","$4,668,600"
1,Cabernet Franc,1999,578.0,101.0,679.0,"1,769.0","$2,047.0","$3,622,400"
2,Cabernet Sauvignon,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,418.0","$79,240,400"
3,Cabernet Sauvignon,1999,"7,625.0","2,298.0","9,923.0","21,037.0","$2,158.0","$45,400,600"
4,Carignane,2000,196.0,1.0,197.0,766.0,"$1,392.0","$1,065,400"
5,Carignane,1999,201.0,1.0,202.0,590.0,"$1,215.0","$717,200"
6,Merlot,2000,"6,564.0","1,058.0","7,622.0","31,480.0","$2,019.0","$63,566,200"
7,Merlot,1999,"6,276.0",848.0,"7,124.0","24,558.0","$1,979.0","$48,599,700"
8,Meunier,2000,129.0,17.0,146.0,637.0,"$2,069.0","$1,318,700"
9,Meunier,1999,129.0,17.0,146.0,426.0,"$1,817.0","$774,807"


In [52]:
red_2000 = correct_col_1999_2000.query('Year == "2000"').reset_index(drop=True)
red_2000.insert(0, 'County', 'Sonoma')
red_2000.insert(2, 'Grape Color', 'Red')
red_2000

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,"2,097.0","$2,227.0","$4,668,600"
1,Sonoma,Cabernet Sauvignon,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,418.0","$79,240,400"
2,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,"$1,392.0","$1,065,400"
3,Sonoma,Merlot,Red,2000,"6,564.0","1,058.0","7,622.0","31,480.0","$2,019.0","$63,566,200"
4,Sonoma,Meunier,Red,2000,129.0,17.0,146.0,637.0,"$2,069.0","$1,318,700"
5,Sonoma,Napa Gamay,Red,2000,84.0,0.0,84.0,195.0,"$1,144.0","$222,500"
6,Sonoma,Petite Sirah,Red,2000,280.0,86.0,365.0,"1,073.0","$2,128.0","$2,284,000"
7,Sonoma,Petite Verdot,Red,2000,86.0,77.0,162.0,282.0,"$2,176.0","$614,000"
8,Sonoma,Pinot Noir,Red,2000,"4,973.0","3,631.0","8,604.0","17,520.0","$2,094.0","$36,692,700"
9,Sonoma,Sangiovese,Red,2000,281.0,68.0,349.0,"1,791.0","$1,860.0","$3,331,000"


In [53]:
sonoma_2001_fixed11_16_06_05_04_03_02_01_00_still_matching = pd.concat([sonoma_2001_fixed11_16_06_05_04_03_02_01, red_2000]).sort_values(['Year', 'Varietal']).drop_duplicates().reset_index(drop=True)
sonoma_2001_fixed11_16_06_05_04_03_02_01_00_still_matching

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,"2,097.0","$2,226.53","$4,668,600"
1,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,"2,097.0","$2,227.0","$4,668,600"
2,Sonoma,Cabernet Sauvigno,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,417.96","$79,240,400"
3,Sonoma,Cabernet Sauvignon,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,418.0","$79,240,400"
4,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,"$1,391.55","$1,065,400"
...,...,...,...,...,...,...,...,...,...,...
251,Sonoma,Pinot Noir,Red,2016,"13,343.1",385.2,"13,728.3","42,566.3","$3,680.50","$156,665,300"
252,Sonoma,Sangiovese,Red,2016,316.9,0.0,316.9,"1,091.1","$2,314.48","$2,525,300"
253,Sonoma,Syrah-Shiraz,Red,2016,"1,575.6",13.5,"1,589.1","3,618.1","$2,596.01","$9,392,600"
254,Sonoma,TOTAL ALL REDS Includes Other Reds,Red,2016,"39,821.4","1,342.0","41,163.5","130,589.3","$3,008.75","$392,910,600"


In [54]:
red_2000_matched = sonoma_2001_fixed11_16_06_05_04_03_02_01_00_still_matching.query('Year == "2000"').drop_duplicates(subset=['Varietal'], keep='first')
red_2000_matched

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,"2,097.0","$2,226.53","$4,668,600"
2,Sonoma,Cabernet Sauvigno,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,417.96","$79,240,400"
3,Sonoma,Cabernet Sauvignon,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,418.0","$79,240,400"
4,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,"$1,391.55","$1,065,400"
6,Sonoma,Merlot,Red,2000,"6,564.0","1,058.0","7,622.0","31,480.0","$2,019.29","$63,566,200"
8,Sonoma,Meunier,Red,2000,129.0,17.0,146.0,637.0,"$2,069.27","$1,318,700"
10,Sonoma,Napa Gamay,Red,2000,84.0,0.0,84.0,195.0,"$1,143.58","$222,500"
12,Sonoma,Other Reds,Red,2000,455.0,319.0,774.0,"1,844.0","$2,036.69","$3,755,200"
14,Sonoma,Petite Sirah,Red,2000,280.0,86.0,366.0,"1,073.0","$2,127.81","$2,284,000"
16,Sonoma,Petite Verdot,Red,2000,86.0,77.0,163.0,282.0,"$2,176.29","$614,000"


In [55]:
sonoma_2001_fixed11_16_06_05_04_03_02_01_00 = pd.concat([sonoma_2001_fixed11_16_06_05_04_03_02_01_00_still_matching.query('Year != "2000"'), red_2000_matched]).sort_values(['Year', 'Varietal']).reset_index(drop=True)
sonoma_2001_fixed11_16_06_05_04_03_02_01_00

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,"2,097.0","$2,226.53","$4,668,600"
1,Sonoma,Cabernet Sauvigno,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,417.96","$79,240,400"
2,Sonoma,Cabernet Sauvignon,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,418.0","$79,240,400"
3,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,"$1,391.55","$1,065,400"
4,Sonoma,Merlot,Red,2000,"6,564.0","1,058.0","7,622.0","31,480.0","$2,019.29","$63,566,200"
...,...,...,...,...,...,...,...,...,...,...
238,Sonoma,Pinot Noir,Red,2016,"13,343.1",385.2,"13,728.3","42,566.3","$3,680.50","$156,665,300"
239,Sonoma,Sangiovese,Red,2016,316.9,0.0,316.9,"1,091.1","$2,314.48","$2,525,300"
240,Sonoma,Syrah-Shiraz,Red,2016,"1,575.6",13.5,"1,589.1","3,618.1","$2,596.01","$9,392,600"
241,Sonoma,TOTAL ALL REDS Includes Other Reds,Red,2016,"39,821.4","1,342.0","41,163.5","130,589.3","$3,008.75","$392,910,600"


## Fixing 2010 Crop Report - we decide to use 2011's 2010 version
### 1. Assumed 2011's TOTAL REDS Row contains TOTAL REDS Including Other Reds 


In [56]:
red_2010 = sonoma_2001_fixed11_16_06_05_04_03_02_01_00.query('Year == "2010"')
red_2010_fixed = red_2010.query('Varietal != "Other Reds" & Varietal != "TOTAL ALL REDS NOT Including Other Reds"')
red_2010_fixed

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
144,Sonoma,Cabernet Franc,Red,2010,653.9,14.3,668.2,"1,436.3","$2,166.55","$3,111,900"
145,Sonoma,Cabernet Sauvignon,Red,2010,"11,549.0",468.3,"12,017.3","37,933.3","$2,083.96","$79,051,500"
146,Sonoma,Carignane,Red,2010,180.3,1.0,181.3,270.4,"$1,949.11","$527,100"
147,Sonoma,Malbec,Red,2010,310.4,69.1,379.5,770.5,"$2,430.83","$1,873,000"
148,Sonoma,Merlot,Red,2010,"5,248.0",55.6,"5,303.6","17,440.9","$1,297.08","$22,622,300"
149,Sonoma,Meunier,Red,2010,108.5,15.0,123.5,406.5,"$2,928.56","$1,190,500"
151,Sonoma,Petite Sirah,Red,2010,522.6,37.7,560.3,"2,085.9","$2,377.94","$4,960,200"
152,Sonoma,Petite Verdot,Red,2010,202.9,9.2,212.1,695.5,"$2,476.49","$1,722,400"
153,Sonoma,Pinot Noir,Red,2010,"11,456.0",675.5,"12,131.5","29,791.3","$2,825.12","$84,164,000"
154,Sonoma,Sangiovese,Red,2010,259.7,48.4,308.1,983.6,"$2,244.09","$2,207,300"


In [57]:
sonoma_2001_fixed11_16_06_05_04_03_02_01_00_10 = pd.concat([sonoma_2001_fixed11_16_06_05_04_03_02_01_00.query('Year != "2010"'), red_2010_fixed]).sort_values(['Year', 'Varietal']).reset_index(drop=True)
sonoma_2001_fixed11_16_06_05_04_03_02_01_00_10

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,"2,097.0","$2,226.53","$4,668,600"
1,Sonoma,Cabernet Sauvigno,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,417.96","$79,240,400"
2,Sonoma,Cabernet Sauvignon,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,418.0","$79,240,400"
3,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,"$1,391.55","$1,065,400"
4,Sonoma,Merlot,Red,2000,"6,564.0","1,058.0","7,622.0","31,480.0","$2,019.29","$63,566,200"
...,...,...,...,...,...,...,...,...,...,...
236,Sonoma,Pinot Noir,Red,2016,"13,343.1",385.2,"13,728.3","42,566.3","$3,680.50","$156,665,300"
237,Sonoma,Sangiovese,Red,2016,316.9,0.0,316.9,"1,091.1","$2,314.48","$2,525,300"
238,Sonoma,Syrah-Shiraz,Red,2016,"1,575.6",13.5,"1,589.1","3,618.1","$2,596.01","$9,392,600"
239,Sonoma,TOTAL ALL REDS Includes Other Reds,Red,2016,"39,821.4","1,342.0","41,163.5","130,589.3","$3,008.75","$392,910,600"


In [58]:
red_2008_updated = sonoma_2001_fixed11_16_06_05_04_03_02_01_00_10.query('Year == "2008"').drop_duplicates(subset=['Varietal'], keep='last')
red_2008_updated

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
115,Sonoma,Cabernet Franc,Red,2008,638.7,49.9,688.6,"1,594.8","$2,338.83","$3,730,000"
116,Sonoma,Cabernet Sauvignon,Red,2008,"11,576.0",678.4,"12,254.4","31,870.8","$2,321.51","$73,988,400"
117,Sonoma,Carignane,Red,2008,181.1,0.0,181.1,327.7,"$1,944.90","$637,400"
118,Sonoma,Malbec,Red,2008,272.0,76.0,348.0,787.6,"$2,550.60","$2,008,900"
119,Sonoma,Merlot,Red,2008,"5,875.6",133.4,"6,009.0","13,298.5","$1,555.48","$20,685,600"
120,Sonoma,Meunier,Red,2008,105.9,4.5,110.4,325.2,"$2,769.47","$900,700"
122,Sonoma,Other Reds,Red,2008,512.7,80.7,593.4,"1,5828.8","$1,619.83","$2,476,400"
123,Sonoma,Petite Sirah,Red,2008,466.4,98.1,564.5,"1,905.9","$2,535.97","$4,833,400"
124,Sonoma,Petite Verdot,Red,2008,174.4,45.1,219.5,650.3,"$2,707.89","$1,761,000"
125,Sonoma,Pinot Noir,Red,2008,"10,293.2","2,242.5","12,535.7","29,441.5","$3,170.71","$93,350,500"


In [59]:
sonoma_2001_fixed11_16_06_05_04_03_02_01_00_10_08 = pd.concat([sonoma_2001_fixed11_16_06_05_04_03_02_01_00_10.query('Year != "2008"'), red_2008_updated]).sort_values(['Year', 'Varietal']).reset_index(drop=True)
sonoma_2001_fixed11_16_06_05_04_03_02_01_00_10_08

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,"2,097.0","$2,226.53","$4,668,600"
1,Sonoma,Cabernet Sauvigno,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,417.96","$79,240,400"
2,Sonoma,Cabernet Sauvignon,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,418.0","$79,240,400"
3,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,"$1,391.55","$1,065,400"
4,Sonoma,Merlot,Red,2000,"6,564.0","1,058.0","7,622.0","31,480.0","$2,019.29","$63,566,200"
...,...,...,...,...,...,...,...,...,...,...
235,Sonoma,Pinot Noir,Red,2016,"13,343.1",385.2,"13,728.3","42,566.3","$3,680.50","$156,665,300"
236,Sonoma,Sangiovese,Red,2016,316.9,0.0,316.9,"1,091.1","$2,314.48","$2,525,300"
237,Sonoma,Syrah-Shiraz,Red,2016,"1,575.6",13.5,"1,589.1","3,618.1","$2,596.01","$9,392,600"
238,Sonoma,TOTAL ALL REDS Includes Other Reds,Red,2016,"39,821.4","1,342.0","41,163.5","130,589.3","$3,008.75","$392,910,600"


In [60]:
# pd.set_option('display.max_rows', 300)

In [61]:
numeric_columns = ['Bearing', 'Non-Bearing', 'Total', 'Tons', '$/Ton', 'Total Value']
year_numeric = ['Year']
df = sonoma_2001_fixed11_16_06_05_04_03_02_01_00_10_08
df[year_numeric] = df[year_numeric].apply(pd.to_numeric)
df_up_to_2009 = df.query('Year <= 2009')
df_up_to_2009[numeric_columns] = df_up_to_2009[numeric_columns].replace('[\$,]', '', regex=True)

# Fix 3453.0* error for 2005 

df_up_to_2009.query('Year == 2005')
df_up_to_2009['Non-Bearing'] = df_up_to_2009['Non-Bearing'].str.replace('*', '', regex=True)

df_up_to_2009[numeric_columns] = df_up_to_2009[numeric_columns].apply(pd.to_numeric)
df_up_to_2009


,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,2097.0,2226.53,4668600
1,Sonoma,Cabernet Sauvigno,Red,2000,7328.0,3688.0,11016.0,32772.0,2417.96,79240400
2,Sonoma,Cabernet Sauvignon,Red,2000,7328.0,3688.0,11016.0,32772.0,2418.00,79240400
3,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,1391.55,1065400
4,Sonoma,Merlot,Red,2000,6564.0,1058.0,7622.0,31480.0,2019.29,63566200
...,...,...,...,...,...,...,...,...,...,...
138,Sonoma,Pinot Noir,Red,2009,10746.2,1988.5,12734.7,31961.2,3043.08,97260500
139,Sonoma,Sangiovese,Red,2009,298.6,11.1,309.7,1210.6,2231.17,2701100
140,Sonoma,Syrah-Shiraz,Red,2009,1816.6,168.5,1985.1,5563.3,2314.65,12877100
141,Sonoma,TOTAL ALL REDS NOT Including Other Reds,Red,2009,37937.2,3636.2,41573.4,120717.0,2430.05,293348400


## Combining Other Reds +TOTAL ALL REDS NOT Including Other Reds = TOTAL ALL REDS Including Other Reds

In [62]:
# pd.set_option('display.max_rows', 300)

In [63]:
grouped = df_up_to_2009.groupby('Year')

new_df = pd.DataFrame()

for year, group in grouped:
    # find the rows to combine 
    total_all_reds_row = group.query('Varietal == "TOTAL ALL REDS NOT Including Other Reds"')
    other_reds_row = group.query('Varietal == "Other Reds"')

    # print(total_all_reds_row)

    # Combine the rows
    combined_row = total_all_reds_row.copy()
    combined_row['Varietal'] = 'TOTAL ALL REDS Including Other Reds'
    combined_row[numeric_columns] += other_reds_row[numeric_columns].values

    # Append the combined row to the new dataframe
    # print(combined_row)
    new_df = pd.concat([new_df, combined_row])

df_up_to_2009_combined = pd.concat([df_up_to_2009, new_df])
df_up_to_2009_combined_updated = df_up_to_2009_combined.query('Varietal != "TOTAL ALL REDS NOT Including Other Reds" & Varietal != "Other Reds"')
df_up_to_2009_combined_updated = df_up_to_2009_combined_updated.sort_values(['Year', 'Varietal']).reset_index(drop=True)


In [64]:
df_up_to_2009_combined_updated

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,2097.0,2226.53,4668600.0
1,Sonoma,Cabernet Sauvigno,Red,2000,7328.0,3688.0,11016.0,32772.0,2417.96,79240400.0
2,Sonoma,Cabernet Sauvignon,Red,2000,7328.0,3688.0,11016.0,32772.0,2418.00,79240400.0
3,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,1391.55,1065400.0
4,Sonoma,Merlot,Red,2000,6564.0,1058.0,7622.0,31480.0,2019.29,63566200.0
...,...,...,...,...,...,...,...,...,...,...
128,Sonoma,Pinot Noir,Red,2009,10746.2,1988.5,12734.7,31961.2,3043.08,97260500.0
129,Sonoma,Sangiovese,Red,2009,298.6,11.1,309.7,1210.6,2231.17,2701100.0
130,Sonoma,Syrah-Shiraz,Red,2009,1816.6,168.5,1985.1,5563.3,2314.65,12877100.0
131,Sonoma,TOTAL ALL REDS Including Other Reds,Red,2009,38464.4,3823.8,42288.2,122556.7,2430.05,299528400.0


In [65]:
df_up_to_2009_combined_updated.head()

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,2097.0,2226.53,4668600.0
1,Sonoma,Cabernet Sauvigno,Red,2000,7328.0,3688.0,11016.0,32772.0,2417.96,79240400.0
2,Sonoma,Cabernet Sauvignon,Red,2000,7328.0,3688.0,11016.0,32772.0,2418.00,79240400.0
3,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,1391.55,1065400.0
4,Sonoma,Merlot,Red,2000,6564.0,1058.0,7622.0,31480.0,2019.29,63566200.0


In [66]:
df_up_to_2009_combined_updated.dtypes

County          object
Varietal        object
Grape Color     object
Year             int64
Bearing        float64
Non-Bearing    float64
Total          float64
Tons           float64
$/Ton          float64
Total Value    float64
dtype: object

# We dropped the rows, now is just turn int back to string, and apply $, ., etc

In [67]:
# Format columns
df_up_to_2009_combined_updated['Bearing'] = df_up_to_2009_combined_updated['Bearing'].apply('{:,.1f}'.format)
df_up_to_2009_combined_updated['Non-Bearing'] = df_up_to_2009_combined_updated['Non-Bearing'].apply('{:,.1f}'.format)
df_up_to_2009_combined_updated['Total'] = df_up_to_2009_combined_updated['Total'].apply('{:,.1f}'.format)
df_up_to_2009_combined_updated['Tons'] = df_up_to_2009_combined_updated['Tons'].apply('{:,.1f}'.format)
df_up_to_2009_combined_updated['$/Ton'] = df_up_to_2009_combined_updated['$/Ton'].apply('${:,.2f}'.format)
df_up_to_2009_combined_updated['Total Value'] = df_up_to_2009_combined_updated['Total Value'].apply('${:,.0f}'.format)
df_up_to_2009_combined_updated

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,"2,097.0","$2,226.53","$4,668,600"
1,Sonoma,Cabernet Sauvigno,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,417.96","$79,240,400"
2,Sonoma,Cabernet Sauvignon,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,418.00","$79,240,400"
3,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,"$1,391.55","$1,065,400"
4,Sonoma,Merlot,Red,2000,"6,564.0","1,058.0","7,622.0","31,480.0","$2,019.29","$63,566,200"
...,...,...,...,...,...,...,...,...,...,...
128,Sonoma,Pinot Noir,Red,2009,"10,746.2","1,988.5","12,734.7","31,961.2","$3,043.08","$97,260,500"
129,Sonoma,Sangiovese,Red,2009,298.6,11.1,309.7,"1,210.6","$2,231.17","$2,701,100"
130,Sonoma,Syrah-Shiraz,Red,2009,"1,816.6",168.5,"1,985.1","5,563.3","$2,314.65","$12,877,100"
131,Sonoma,TOTAL ALL REDS Including Other Reds,Red,2009,"38,464.4","3,823.8","42,288.2","122,556.7","$2,430.05","$299,528,400"


In [68]:
cleaned_sonoma_beyond_2010_2016 = sonoma_2001_fixed11_16_06_05_04_03_02_01_00_10_08.query('Year > 2009')
cleaned_sonoma_beyond_2010_2016

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
143,Sonoma,Cabernet Franc,Red,2010,653.9,14.3,668.2,"1,436.3","$2,166.55","$3,111,900"
144,Sonoma,Cabernet Sauvignon,Red,2010,"11,549.0",468.3,"12,017.3","37,933.3","$2,083.96","$79,051,500"
145,Sonoma,Carignane,Red,2010,180.3,1.0,181.3,270.4,"$1,949.11","$527,100"
146,Sonoma,Malbec,Red,2010,310.4,69.1,379.5,770.5,"$2,430.83","$1,873,000"
147,Sonoma,Merlot,Red,2010,"5,248.0",55.6,"5,303.6","17,440.9","$1,297.08","$22,622,300"
...,...,...,...,...,...,...,...,...,...,...
235,Sonoma,Pinot Noir,Red,2016,"13,343.1",385.2,"13,728.3","42,566.3","$3,680.50","$156,665,300"
236,Sonoma,Sangiovese,Red,2016,316.9,0.0,316.9,"1,091.1","$2,314.48","$2,525,300"
237,Sonoma,Syrah-Shiraz,Red,2016,"1,575.6",13.5,"1,589.1","3,618.1","$2,596.01","$9,392,600"
238,Sonoma,TOTAL ALL REDS Includes Other Reds,Red,2016,"39,821.4","1,342.0","41,163.5","130,589.3","$3,008.75","$392,910,600"


In [69]:
sonoma_2000_2016_cleaned = pd.concat([df_up_to_2009_combined_updated, cleaned_sonoma_beyond_2010_2016])
sonoma_2000_2016_cleaned.dtypes

County         object
Varietal       object
Grape Color    object
Year            int64
Bearing        object
Non-Bearing    object
Total          object
Tons           object
$/Ton          object
Total Value    object
dtype: object

In [70]:
sonoma_2000_2016_cleaned

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,"2,097.0","$2,226.53","$4,668,600"
1,Sonoma,Cabernet Sauvigno,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,417.96","$79,240,400"
2,Sonoma,Cabernet Sauvignon,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,418.00","$79,240,400"
3,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,"$1,391.55","$1,065,400"
4,Sonoma,Merlot,Red,2000,"6,564.0","1,058.0","7,622.0","31,480.0","$2,019.29","$63,566,200"
...,...,...,...,...,...,...,...,...,...,...
235,Sonoma,Pinot Noir,Red,2016,"13,343.1",385.2,"13,728.3","42,566.3","$3,680.50","$156,665,300"
236,Sonoma,Sangiovese,Red,2016,316.9,0.0,316.9,"1,091.1","$2,314.48","$2,525,300"
237,Sonoma,Syrah-Shiraz,Red,2016,"1,575.6",13.5,"1,589.1","3,618.1","$2,596.01","$9,392,600"
238,Sonoma,TOTAL ALL REDS Includes Other Reds,Red,2016,"39,821.4","1,342.0","41,163.5","130,589.3","$3,008.75","$392,910,600"


In [71]:
sonoma_2017_sorted = sonoma_2017.sort_values(['Year', 'Varietal'])
sonoma_2017_sorted.query('Year == "2016"')

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
71,Sonoma,Cabernet Franc,Red,2016,578.5,28.5,607.0,"1,382.5","$2,955.14","$4,085,500"
72,Sonoma,Cabernet Sauvignon,Red,2016,"12,456.7",560.0,"13,016.7","41,575.7","$2,964.60","$123,255,300"
73,Sonoma,Carignane,Red,2016,162.7,0.0,162.7,305.8,"$2,195.62","$671,500"
74,Sonoma,Grenache,Red,2016,152.7,3.6,156.3,739.7,"$2,995.16","$2,215,500"
75,Sonoma,Malbec,Red,2016,551.5,4.7,556.2,"2,960.5","$2,629.78","$7,785,500"
76,Sonoma,Mataro/Mouvedere,Red,2016,41.6,0.9,42.5,771.5,"$2,119.36","$1,635,100"
77,Sonoma,Merlot,Red,2016,"4,115.7",24.6,"4,140.3","13,761.0","$1,818.10","$25,018,900"
79,Sonoma,Petit Verdot,Red,2016,278.3,16.5,294.8,"1,332.0","$2,976.71","$3,965,000"
78,Sonoma,Petite Sirah,Red,2016,694.5,4.2,698.7,"3,266.3","$2,797.89","$9,138,700"
80,Sonoma,Pinot Noir,Red,2016,"13,343.1",385.2,"13,728.3","42,566.3","$3,680.50","$156,665,300"


In [72]:
sonoma_2016_cleaned = pd.concat([sonoma_2000_2016_cleaned.query('Year == "2016"'), sonoma_2017_sorted.query('Year == "2016"')]).sort_values('Varietal').reset_index(drop=True).drop_duplicates()
sonoma_2016_cleaned

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2016,578.5,28.5,607.0,"1,382.5","$2,955.14","$4,085,500"
1,Sonoma,Cabernet Sauvignon,Red,2016,"12,456.7",560.0,"13,016.7","41,575.7","$2,964.60","$123,255,300"
2,Sonoma,Carignane,Red,2016,162.7,0.0,162.7,305.8,"$2,195.62","$671,500"
3,Sonoma,Grenache,Red,2016,152.7,3.6,156.3,739.7,"$2,995.16","$2,215,500"
4,Sonoma,Malbec,Red,2016,551.5,4.7,556.2,"2,960.5","$2,629.78","$7,785,500"
5,Sonoma,Mataro/Mouvedere,Red,2016,41.6,0.9,42.5,771.5,"$2,119.36","$1,635,100"
6,Sonoma,Merlot,Red,2016,"4,115.7",24.6,"4,140.3","13,761.0","$1,818.10","$25,018,900"
7,Sonoma,Petit Verdot,Red,2016,278.3,16.5,294.8,"1,332.0","$2,976.71","$3,965,000"
8,Sonoma,Petite Sirah,Red,2016,694.5,4.2,698.7,"3,266.3","$2,797.89","$9,138,700"
9,Sonoma,Pinot Noir,Red,2016,"13,343.1",385.2,"13,728.3","42,566.3","$3,680.50","$156,665,300"


In [73]:
sonoma_2016_2021 = pd.concat([sonoma_2017_sorted.query('Year != "2016"'), sonoma_2016_cleaned]).sort_values(['Year','Varietal']).reset_index(drop=True)
sonoma_2016_2021

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2016,578.5,28.5,607.0,"1,382.5","$2,955.14","$4,085,500"
1,Sonoma,Cabernet Sauvignon,Red,2016,"12,456.7",560.0,"13,016.7","41,575.7","$2,964.60","$123,255,300"
2,Sonoma,Carignane,Red,2016,162.7,0.0,162.7,305.8,"$2,195.62","$671,500"
3,Sonoma,Grenache,Red,2016,152.7,3.6,156.3,739.7,"$2,995.16","$2,215,500"
4,Sonoma,Malbec,Red,2016,551.5,4.7,556.2,"2,960.5","$2,629.78","$7,785,500"
...,...,...,...,...,...,...,...,...,...,...
81,Sonoma,Pinot Noir,Red,2021,"12,608.0",265.0,"12,873.0","48,496.5","$3,512.27","$170,332,800"
82,Sonoma,Sangiovese,Red,2021,278.0,4.0,282.0,879.7,"$2,493.41","$2,193,500"
83,Sonoma,Syrah-Shiraz,Red,2021,"1,326.0",34.0,"1,360.0","2,450.6","$2,871.85","$7,037,800"
84,Sonoma,TOTAL ALL REDS Includes Other Reds,Red,2021,"38,128.0","1,056.0","39,184.0","121,693.0","$3,060.13","$372,396,400"


In [74]:
# pd.set_option('display.max_rows', 400)

In [75]:
sonoma_red_cleaned = pd.concat([sonoma_2000_2016_cleaned.query('Year != "2016"'), sonoma_2016_2021]).sort_values(['Year','Varietal']).reset_index(drop=True)
sonoma_red_cleaned

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,"2,097.0","$2,226.53","$4,668,600"
1,Sonoma,Cabernet Sauvigno,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,417.96","$79,240,400"
2,Sonoma,Cabernet Sauvignon,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,418.00","$79,240,400"
3,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,"$1,391.55","$1,065,400"
4,Sonoma,Merlot,Red,2000,"6,564.0","1,058.0","7,622.0","31,480.0","$2,019.29","$63,566,200"
...,...,...,...,...,...,...,...,...,...,...
311,Sonoma,Pinot Noir,Red,2021,"12,608.0",265.0,"12,873.0","48,496.5","$3,512.27","$170,332,800"
312,Sonoma,Sangiovese,Red,2021,278.0,4.0,282.0,879.7,"$2,493.41","$2,193,500"
313,Sonoma,Syrah-Shiraz,Red,2021,"1,326.0",34.0,"1,360.0","2,450.6","$2,871.85","$7,037,800"
314,Sonoma,TOTAL ALL REDS Includes Other Reds,Red,2021,"38,128.0","1,056.0","39,184.0","121,693.0","$3,060.13","$372,396,400"


In [76]:
sonoma_red_cleaned_copy = sonoma_red_cleaned.copy()
sonoma_red_cleaned_copy.head(15)

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,"2,097.0","$2,226.53","$4,668,600"
1,Sonoma,Cabernet Sauvigno,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,417.96","$79,240,400"
2,Sonoma,Cabernet Sauvignon,Red,2000,"7,328.0","3,688.0","11,016.0","32,772.0","$2,418.00","$79,240,400"
3,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,"$1,391.55","$1,065,400"
4,Sonoma,Merlot,Red,2000,"6,564.0","1,058.0","7,622.0","31,480.0","$2,019.29","$63,566,200"
5,Sonoma,Meunier,Red,2000,129.0,17.0,146.0,637.0,"$2,069.27","$1,318,700"
6,Sonoma,Napa Gamay,Red,2000,84.0,0.0,84.0,195.0,"$1,143.58","$222,500"
7,Sonoma,Petite Sirah,Red,2000,280.0,86.0,366.0,"1,073.0","$2,127.81","$2,284,000"
8,Sonoma,Petite Verdot,Red,2000,86.0,77.0,163.0,282.0,"$2,176.29","$614,000"
9,Sonoma,Pinot Noir,Red,2000,"4,973.0","3,631.0","8,604.0","17,520.0","$2,094.39","$36,692,700"


In [77]:
for year in sonoma_red_cleaned_copy['Year'].unique():
    # Identify the rows where Varietal is 'Zinfandel' for the current year
    zinfandel_rows = (sonoma_red_cleaned_copy['Varietal'] == 'Zinfandel') & (sonoma_red_cleaned_copy['Year'] == year)

    # Get the indices of the Zinfandel rows
    zinfandel_indices = sonoma_red_cleaned_copy[zinfandel_rows].index
    # print(zinfandel_indices)

    # Iterate over the Zinfandel rows
    for zinfandel_index in zinfandel_indices:
        # Find the previous row index
        # print(zinfandel_index)
        prev_row_index = zinfandel_index - 1

        # Swap the rows
        sonoma_red_cleaned_copy.iloc[[zinfandel_index, prev_row_index]] = sonoma_red_cleaned_copy.iloc[[prev_row_index, zinfandel_index]].values

# Reset the index

sonoma_red_cleaned_copy.reset_index(drop=True, inplace=True)

In [78]:
numeric_columns = ['Year','Bearing', 'Non-Bearing', 'Total', 'Tons', '$/Ton', 'Total Value']
year_numeric = ['Year']
df = sonoma_red_cleaned_copy
df[year_numeric] = df[year_numeric].astype(int)
df[numeric_columns] = sonoma_red_cleaned_copy[numeric_columns].replace('[\$,]', '', regex=True)
df

,County,Varietal,Grape Color,Year,Bearing,Non-Bearing,Total,Tons,$/Ton,Total Value
0,Sonoma,Cabernet Franc,Red,2000,523.0,151.0,674.0,2097.0,2226.53,4668600
1,Sonoma,Cabernet Sauvigno,Red,2000,7328.0,3688.0,11016.0,32772.0,2417.96,79240400
2,Sonoma,Cabernet Sauvignon,Red,2000,7328.0,3688.0,11016.0,32772.0,2418.00,79240400
3,Sonoma,Carignane,Red,2000,196.0,1.0,197.0,766.0,1391.55,1065400
4,Sonoma,Merlot,Red,2000,6564.0,1058.0,7622.0,31480.0,2019.29,63566200
...,...,...,...,...,...,...,...,...,...,...
311,Sonoma,Pinot Noir,Red,2021,12608.0,265.0,12873.0,48496.5,3512.27,170332800
312,Sonoma,Sangiovese,Red,2021,278.0,4.0,282.0,879.7,2493.41,2193500
313,Sonoma,Syrah-Shiraz,Red,2021,1326.0,34.0,1360.0,2450.6,2871.85,7037800
314,Sonoma,Zinfandel,Red,2021,4613.0,116.0,4729.0,11660.0,3113.73,36306100


In [79]:
# df.to_csv('model_variables_param_table.csv', index=False)

## Indivdual Year Testing

In [ ]:
#2011 is urls[10]
#2010 is urls[11]
#2009 is urls[12]
#2008 is urls[13]
#2007 is urls[14]
#2006 is urls[15]
#2005 is urls[16]
#2004 is urls[17]
#2003 is urls[18]
#2002 is urls[19]
#2001 is urls[20]
#2000 is urls[21]



urls[18]

In [ ]:
# utilize camelot to read pdf file at page 12, capture precisely with stream and close up the row gap with row_tol = 10

tables = camelot.read_pdf(urls[18], pages=page, flavor='stream', row_tol=8, split_text=True)
data = tables[0].df
dat = tables[0].parsing_report
dat

In [ ]:
camelot.plot(tables[0], kind='contour').show()

In [ ]:
data

In [ ]:
# data.columns = ['0', '1', '2', '3', '4', '5', '6', '7']
# data

In [ ]:
# drop rows that is not helpful
data = data.drop([0,1,2], axis=0)
data
# for 2011 report, also drop an empty row 

In [ ]:
# # For 2016 crop report:

# # temp is non-bearing
# data['Temp'] = data['3']  # Create a temporary column and store 'Bearing' values
# data['3'] = data['2']  # Move 'Non-Bearing' values to 'Bearing' column
# data['2'] = data['Temp']  # Move temporary values to 'Non-Bearing' column
# data.drop('Temp', axis=1, inplace=True)  # Drop the temporary column
# data

In [ ]:
# special for 2006, 2002; $ sign is in its separate column
# rename its column, add the dollar column with the value
# data.columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


# data['7'] = data['6'] + data['7']
# data['9'] = data['8'] + data['9']
# data = data.drop(['6', '8'], axis=1)
# data = data[:28]
# data

In [ ]:
# special for 2010; $ sign is in its separate column
# rename its column, add the dollar column with the value
# data.columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


# data['7'] = data['6'] + data['7']
# data['9'] = data['8'] + data['9']
# data = data.drop(['6', '8'], axis=1)
# data

In [ ]:
data = data[:28]
data.loc[28][0] = data.loc[29][0]
data.loc[29][0] = ''

In [ ]:
data

In [ ]:
# rename my columns
new_columns = ['Varietal', 'Year', 'Bearing' , 'Non-Bearing', 'Total', 'Tons', '$/Ton', 'Total Value']
data = data.rename(columns=dict(zip(data.columns,new_columns)))
data

In [ ]:
if '.' in data['Tons'].str.rstrip('0'):
    data['Tons'].str.rstrip('0')

In [ ]:
trailing_zero_strip = data['Tons'].str.rstrip('0')
trailing_zero_strip.replace('19,571.', '19,571.0')

In [ ]:
trailing_zero_strip[20]

In [ ]:
# data['Bearing'] = data['Bearing'].str.replace(' .', '.', regex=False)
# data['Bearing'] = data['Bearing'].str.replace('--', '0.0', regex=False)
# data['Non-Bearing'] = data['Non-Bearing'].str.replace(' .', '.', regex=False)
# data['Non-Bearing'] = data['Non-Bearing'].str.replace('--', '0.0', regex=False)

# data['Total'] = data['Total'].str.replace(' .', '.', regex=False)
# data['Total'] = data['Total'].str.replace('--', '0.0', regex=False)
# data['$/Ton'] = data['$/Ton'].str.replace(' .', '.', regex=False)
# data['Total Value'] = data['Total Value'].str.replace(' ', ',')

In [ ]:
# data['Total'] = pd.to_numeric(data['Total'], errors='coerce')
# data

In [ ]:
# data = data[:28]
# data

In [ ]:
# data.loc[28][0] = data.loc[29][0]
# data.loc[29][0] = ''

In [ ]:

# specific fixes for 2004

# 28		2010	37,804.4	1,606.3	39,410.7	108,481.0\n(a	) $2,255.85 (a	) $244,717,100
# 29	TOTAL REDS	2009	37,937.2	3,636.2	41,573.4	120,717.0\n(a	) $2,430.05 (a	) $293,348,400	
			
# data.loc[28][0] = data.loc[29][0]
# data.loc[29][0] = ''

# # data drop empty row
# data = data.query("Year != '' and Total != '' and Tons != ''")
# data

# data drop empty row
# data = data[:28]
# data

In [ ]:
# data['Varietal'].unique()

In [ ]:
# data.loc[data['Varietal'].str.contains('Cabernet'), 'Varietal'] = 'Cabernet Sauvignon'
# data.loc[data['Varietal'].str.contains('Sangiovese/'), 'Varietal'] = 'Sangiovese/Sangioveto'

In [ ]:
# data['Varietal'].str.contains('Sangiovese/')

In [ ]:
data['Varietal'] = data['Varietal'].replace('Mataro/', 'Mataro/Mouvedere')
data['Varietal']= data['Varietal'].replace('Cabernet', 'Cabernet Sauvignon')

# for 2016 only 
data['Varietal'] = data['Varietal'].replace('Sangiovese/', 'Sangiovese')
# for 2016 Syrah - Shiraz to Syrah-Shiraz ; 
data['Varietal'] = data['Varietal'].replace('Syrah - Shiraz', 'Syrah-Shiraz')

# for 2005 only
data['Varietal'] = data['Varietal'].replace('Cabernet Sauv.', 'Cabernet Sauvignon')

data['Varietal']= data['Varietal'].replace(['', 'including other reds', 'Mouvedere', 'Mourvedere (a)', 'Sauvignon', 'Sangioveto'], np.nan)
data = data.reset_index(drop=True)

data['Varietal'].unique()

In [ ]:
data['Varietal'] = data['Varietal'].dropna().repeat(2).reset_index(drop=True)
data['County'] = 'Sonoma'

In [ ]:
first_column = data.pop('County')
data['Grape Color'] = 'Red'
third_column = data.pop('Grape Color')
data.insert(0, 'County', first_column)
data.insert(2, 'Grape Color', third_column)
data

In [ ]:
data['Tons'] 
data['$/Ton']
data['Total Value']

In [ ]:
data['Bearing'] + '.0'

In [ ]:
# for 2004 only 

data['Bearing'] = data['Bearing'].str.replace(' ', '.')
data['Non-Bearing'] = data['Non-Bearing'].str.replace(' ', '.')
data['Total'] = data['Total'].str.replace(' ', '.')

# data['Tons'] = data['Tons'].str.replace(' ', '.')
# data['$/Ton'] = data['$/Ton'].str.replace(' ', '')
# data['Total Value'] = data['Total Value'].str.replace(' ', ',')

# # specific for 2004

# we could do .apply lambda function for all the possible combinations so we don't have to add specificly for each year

data['$/Ton'] = data['$/Ton'].str.replace(' \n', '', regex=False)
data['$/Ton'] = data['$/Ton'].str.replace(' ', '', regex=False)
data['Total Value'] = data['Total Value'].str.replace('  \n', '', regex=False)
data['Total Value'] = data['Total Value'].str.replace(' ', '', regex=False)
data['Total Value'] = data['Total Value'].apply(lambda x: '$' + x if '$' not in x else x)
data

In [ ]:
# # for 2007 only 

# data['Bearing'] = data['Bearing'].str.replace(' ', '.')
# data['Non-Bearing'] = data['Non-Bearing'].str.replace(' ', '.')
# data['Total'] = data['Total'].str.replace(' ', '.')

# # data['Tons'] = data['Tons'].str.replace(' ', '.')
# # data['$/Ton'] = data['$/Ton'].str.replace(' ', '')
# # data['Total Value'] = data['Total Value'].str.replace(' ', ',')

# # # specific for 2006


# # we could do .apply lambda function for all the possible combinations so we don't have to add specificly for each year

# data['Bearing'] = data['Bearing'].str.replace('*', '', regex=False)
# data['Total'] = data['Total'].str.replace('*', '', regex=False)
# data['$/Ton'] = data['$/Ton'].str.replace(' (a)', '', regex=False)
# data['Total Value'] = data['Total Value'].str.replace(' \n', '', regex=False)
# data['Total Value'] = data['Total Value'].str.replace(' ', '', regex=False)
# data['Total Value'] = data['Total Value'].apply(lambda x: '$' + x if '$' not in x else x)
# data

In [ ]:
# # for 2010, 2009, 2008 only 

# data['Bearing'] = data['Bearing'].str.replace(' ', '.')
# data['Non-Bearing'] = data['Non-Bearing'].str.replace(' ', '.')
# data['Total'] = data['Total'].str.replace(' ', '.')

# # data['Tons'] = data['Tons'].str.replace(' ', '.')
# # data['$/Ton'] = data['$/Ton'].str.replace(' ', '')
# # data['Total Value'] = data['Total Value'].str.replace(' ', ',')

# # # specific for 2010, 2009, 2008
# data['Tons'] = data['Tons'].str.replace('\n(a', '', regex=False)
# # data['$/Ton'] = data['$/Ton'].str.replace('', '0.0', regex=False)

# data['$/Ton'] = data['$/Ton'].str.replace(') ', '', regex=False)
# data['$/Ton'] = data['$/Ton'].str.replace(' (a', '', regex=False)
# data['$/Ton']  = data['$/Ton'].replace('', '$0.0', regex=False)
# data['Total Value'] = data['Total Value'].str.replace(') ', '', regex=False)

# data['Total Value'] = data['Total Value'].apply(lambda x: '$' + x if '$' not in x else x)
# data

In [ ]:
# data['$/Ton'].replace('', '$0.0', regex=False)

In [ ]:
# for 2011 only 

# data['Bearing'] = data['Bearing'].str.replace(' ', '.')
# data['Non-Bearing'] = data['Non-Bearing'].str.replace(' ', '.')
# data['Total'] = data['Total'].str.replace(' ', '.')
# data['Tons'] = data['Tons'].str.replace(' ', '.')
# data['$/Ton'] = data['$/Ton'].str.replace(' ', '')
# # data['Total Value'] = data['Total Value'].str.replace(' ', ',')

# # specific for 2011
# data['Tons'] = data['Tons'].str.replace('*', '', regex=False)
# data['$/Ton'] = data['$/Ton'].str.replace('*', '', regex=False)
# data['Total Value'] = data['Total Value'].str.replace('\n', '', regex=False)
# data['Total Value'] = data['Total Value'].str.replace('(a)  $ ', '$', regex=False)
# data['Total Value'] = data['Total Value'].str.replace('*', '', regex=False)

In [ ]:
data

In [ ]:
# for 2016 only 

# data['Bearing'].str.replace(' .', '.', regex=False)
# data['Bearing'].str.replace('--', '0.0', regex=False)
# data['Non-Bearing'].str.replace(' .', '.', regex=False)
# data['Non-Bearing'].str.replace('--', '0.0', regex=False)

# data['Total'].str.replace(' .', '.', regex=False)
# data['Total'].str.replace('--', '0.0', regex=False)
# data['$/Ton'].str.replace(' .', '.', regex=False)
# data['Total Value'].str.replace(' ', ',')

In [ ]:
len(data)

In [ ]:
data.to_csv('example.csv', index=False, encoding='utf-8')

In [ ]:
# data['Varietal'] = data['Varietal'].replace('', method='ffill')
# data